<div align="right">
Massimo Nocentini<br>
<br>September 6, 2016: pretty printer with filters
<br>September 5, 2016: big bang
</div>
<br>
<div align="center">
<b>Abstract</b><br>
This document collects some initial interaction with the <a href="http://oeis.org/">OEIS</a>.<br>
We aim to provide an <i>async</i> interface and a <i>pretty printer</i> for search results provided by OEIS.
</div>

# Pretty Printer

The following code cell takes ideas from this [Stack Overflow][stack:overflow] code review:

[stack:overflow]:http://codereview.stackexchange.com/questions/120102/an-oeis-lookup-tool-in-python

In [154]:
from requests import get
from types import SimpleNamespace

In [155]:
code=45

payload = {"fmt": "json", 
           "q": "id:A{:06d}".format(int(code))}

doc_result = get("https://oeis.org/search", params=payload,)
doc = doc_result.json()

In [156]:
doc_result.url

'https://oeis.org/search?q=id%3AA000045&fmt=json'

In [157]:
obj = SimpleNamespace(**doc)

In [158]:
obj.results[0]

{'author': '_N. J. A. Sloane_, 1964',
 'comment': ["Also sometimes called Lamé's sequence.",
  "F(n+2) = number of binary sequences of length n that have no consecutive 0's.",
  'F(n+2) = number of subsets of {1,2,...,n} that contain no consecutive integers.',
  'F(n+1) = number of tilings of a 2 X n rectangle by 2 X 1 dominoes.',
  'F(n+1) = number of matchings (i.e., Hosoya index) in a path graph on n vertices: F(5)=5 because the matchings of the path graph on the vertices A, B, C, D are the empty set, {AB}, {BC}, {CD} and {AB, CD}. - _Emeric Deutsch_, Jun 18 2001',
  'F(n) = number of compositions of n+1 with no part equal to 1. [Cayley, Grimaldi]',
  'Positive terms are the solutions to z = 2*x*y^4 + (x^2)*y^3 - 2*(x^3)*y^2 - y^5 - (x^4)*y + 2*y for x,y >= 0 (Ribenboim, page 193). When x=F(n), y=F(n + 1) and z>0 then z=F(n + 1).',
  'For Fibonacci search see Knuth, Vol. 3; Horowitz and Sahni; etc.',
  "F(n) is the diagonal sum of the entries in Pascal's triangle at 45 degrees slope

In [159]:
obj.results[0].keys()

dict_keys(['comment', 'maple', 'example', 'time', 'mathematica', 'references', 'reference', 'keyword', 'formula', 'xref', 'revision', 'data', 'name', 'number', 'id', 'program', 'offset', 'created', 'link', 'author'])

In [195]:
from IPython.display import Markdown
import re

def pretty_print(doc):
    
    
    head = r"<div align='center'>**<a href='http://oeis.org/A{:06d}'>A{:06d}</a>**: _{}_<br>by {}</div>".format(
        doc['number'], doc['number'], doc['name'], doc['author'])
    keyword = "\n_Keywords_: `{}`".format(doc['keyword'])
    
    upper_limit = 20
    seq = doc['data'].split(',')[:upper_limit]
    array_template = r'''
$$
\begin{env}{{c|{nel}}}
n & {nat} \\
\hline
{id}(n) & {seq}
\end{env}
$$
'''.format(env="{array}",
           nel='c'*len(seq), 
           nat=" & ".join([str(i) for i in range(int(doc['offset'].split(',')[0]), len(seq))]),
           id="A{:06d}".format(doc['number']), 
           seq = " & ".join(seq))
    data = "\n_Data_:\n{}".format(array_template) # nice to build a tabular representation
    
    comment = ("\n_Comments_:\n   - " + "\n   - ".join(doc['comment'])) if 'comment' in doc else ""
    formula = ("\n_Formulae_:\n   - " + "\n   - ".join(doc['formula'])) if 'formula' in doc else ""

    re_pattern = re.compile('(?P<id>A\d{6,6})')
    xref = "\n_Cross references_:\n   - " + "\n   - ".join(
        [re_pattern.sub(r'<a href="http://oeis.org/\g<id>">\g<id></a>',ref) 
         for ref in doc['xref']])
    
    link = ("\n_Links_:\n   - " + "\n   - ".join(doc['link'])) if 'link' in doc else ""
    reference = ("\n_References_:\n   - " + "\n   - ".join(doc['reference'])) if 'reference' in doc else ""
    
    descr = "\n".join([head, keyword, data, comment, formula, xref, link, reference])
    return descr

In [196]:
Markdown(pretty_print(doc['results'][0]))

<div align='center'>**<a href='http://oeis.org/A000045'>A000045</a>**: _Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1._<br>by _N. J. A. Sloane_, 1964</div>

_Keywords_: `core,nonn,nice,easy,hear,changed`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377 & 610 & 987 & 1597 & 2584 & 4181
\end{array}
$$


_Comments_:
   - Also sometimes called Lamé's sequence.
   - F(n+2) = number of binary sequences of length n that have no consecutive 0's.
   - F(n+2) = number of subsets of {1,2,...,n} that contain no consecutive integers.
   - F(n+1) = number of tilings of a 2 X n rectangle by 2 X 1 dominoes.
   - F(n+1) = number of matchings (i.e., Hosoya index) in a path graph on n vertices: F(5)=5 because the matchings of the path graph on the vertices A, B, C, D are the empty set, {AB}, {BC}, {CD} and {AB, CD}. - _Emeric Deutsch_, Jun 18 2001
   - F(n) = number of compositions of n+1 with no part equal to 1. [Cayley, Grimaldi]
   - Positive terms are the solutions to z = 2*x*y^4 + (x^2)*y^3 - 2*(x^3)*y^2 - y^5 - (x^4)*y + 2*y for x,y >= 0 (Ribenboim, page 193). When x=F(n), y=F(n + 1) and z>0 then z=F(n + 1).
   - For Fibonacci search see Knuth, Vol. 3; Horowitz and Sahni; etc.
   - F(n) is the diagonal sum of the entries in Pascal's triangle at 45 degrees slope. - _Amarnath Murthy_, Dec 29 2001
   - F(n+1) is the number of perfect matchings in ladder graph L_n = P_2 X P_n. - Sharon Sela (sharonsela(AT)hotmail.com), May 19 2002
   - F(n+1) = number of (3412,132)-, (3412,213)- and (3412,321)-avoiding involutions in S_n.
   - This is also the Horadam sequence (0,1,1,1). - _Ross La Haye_, Aug 18 2003
   - An INVERT transform of A019590. INVERT([1,1,2,3,5,8,...]) gives A000129. INVERT([1,2,3,5,8,13,21,...]) gives A028859. - _Antti Karttunen_, Dec 12 2003
   - Number of meaningful differential operations of the k-th order on the space R^3. - _Branko Malesevic_, Mar 02 2004
   - F(n)=number of compositions of n-1 with no part greater than 2. Example: F(4)=3 because we have 3 = 1+1+1 = 1+2 = 2+1.
   - F(n) = number of compositions of n into odd parts; e.g., F(6) counts 1+1+1+1+1+1, 1+1+1+3, 1+1+3+1, 1+3+1+1, 1+5, 3+1+1+1, 3+3, 5+1. - _Clark Kimberling_, Jun 22 2004
   - F(n) = number of binary words of length n beginning with 0 and having all runlengths odd; e.g., F(6) counts 010101, 010111, 010001, 011101, 011111, 000101, 000111, 000001. - _Clark Kimberling_, Jun 22 2004
   - The number of sequences (s(0),s(1),...,s(n)) such that 0<s(i)<5, |s(i)-s(i-1)|=1 and s(0)=1 is F(n+1); e.g., F(5+1) = 8 corresponds to 121212, 121232, 121234, 123212, 123232, 123234, 123432, 123434. - _Clark Kimberling_, Jun 22 2004 [corrected by Neven Juric, Jan 09 2009]
   - Likewise F(6+1) = 13 corresponds to these thirteen sequences with seven numbers: 1212121, 1212123, 1212321, 1212323, 1212343, 1232121, 1232123, 1232321, 1232323, 1232343, 1234321, 1234323, 1234343. - Neven Juric, Jan 09 2008
   - A relationship between F(n) and the Mandelbrot set is discussed in the link "Le nombre d'or dans l'ensemble de Mandelbrot" (in French). - _Gerald McGarvey_, Sep 19 2004
   - For n>0, the continued fraction for F(2n-1)*Phi=[F(2n);L(2n-1),L(2n-1),L(2n-1),...] and the continued fraction for F(2n)*Phi=[F(2n+1)-1;1,L(2n)-2,1,L(2n)-2,...]. Also true: F(2n)*Phi=[F(2n+1);-L(2n),L(2n),-L(2n),L(2n),...] where L(i) is the i-th Lucas number (A000204).... - _Clark Kimberling_, Nov 28 2004 [corrected by _Hieronymus Fischer_, Oct 20 2010]
   - F(n+1) (for n>=1) = number of permutations p of 1,2,3,...,n such that |k-p(k)|<=1 for k=1,2,...,n. (For <=2 and <=3, see A002524 and A002526.) - _Clark Kimberling_, Nov 28 2004
   - The ratios F(n+1)/F(n) for n>0 are the convergents to the simple continued fraction expansion of the golden section. - _Jonathan Sondow_, Dec 19 2004
   - Lengths of successive words (starting with a) under the substitution: {a -> ab, b -> a}. - _Jeroen F.J. Laros_, Jan 22 2005
   - The Fibonacci sequence, like any additive sequence, naturally tends to be geometric with common ratio not a rational power of 10; consequently, for a sufficiently large number of terms, Benford's law of first significant digit (i.e., first digit 1 <= d <= 9 occurring with probability log_10(d+1) - log_10(d)) holds. - _Lekraj Beedassy_, Apr 29 2005
   - a(n) = Sum(abs(A108299(n, k)): 0 <= k <= n). - _Reinhard Zumkeller_, Jun 01 2005
   - a(n) = A001222(A000304(n)).
   - Fib(n+2) = Sum_{k=0..n} binomial(floor((n+k)/2),k), row sums of A046854. - _Paul Barry_, Mar 11 2003
   - Number of order ideals of the "zig-zag" poset. See vol. 1, ch. 3, prob. 23 of Stanley. - _Mitch Harris_, Dec 27 2005
   - F(n+1)/F(n) is also the Farey fraction sequence (see A097545 for explanation) for the golden ratio, which is the only number whose Farey fractions and continued fractions are the same. - _Joshua Zucker_, May 08 2006
   - a(n+2) is the number of paths through 2 plates of glass with n reflections (reflections occurring at plate/plate or plate/air interfaces). Cf. A006356-A006359. - _Mitch Harris_, Jul 06 2006
   - F(n+1) equals the number of downsets (i.e., decreasing subsets) of an n-element fence, i.e., an ordered set of height 1 on {1,2,...,n} with 1 > 2 < 3 > 4 < ... n and no other comparabilities. Alternatively, F(n+1) equals the number of subsets A of {1,2,...,n} with the property that, if an odd k is in A, then the adjacent elements of {1,2,...,n} belong to A, i.e., both k - 1 and k + 1 are in A (provided they are in {1,2,...,n}). - _Brian Davey_, Aug 25 2006
   - Number of Kekulé structures in polyphenanthrenes. See the paper by Lukovits and Janezic for details. - _Parthasarathy Nambi_, Aug 22 2006
   - Inverse: With phi = (sqrt(5) + 1)/2, round(log_phi(sqrt((sqrt(5) a(n) + sqrt(5 a(n)^2 - 4))(sqrt(5) a(n) + sqrt(5 a(n)^2 + 4)))/2)) = n for n >= 3, obtained by rounding the arithmetic mean of the inverses given in A001519 and A001906. - David W. Cantrell (DWCantrell(AT)sigmaxi.net), Feb 19 2007
   - A result of Jacobi from 1848 states that every symmetric matrix over a p.i.d. is congruent to a triple-diagonal matrix. Consider the maximal number T(n) of summands in the determinant of an n X n triple-diagonal matrix. This is the same as the number of summands in such a determinant in which the main-, sub- and super-diagonal elements are all nonzero. By expanding on the first row we see that the sequence of T(n)'s is the Fibonacci sequence without the initial stammer on the 1's. - Larry Gerstein (gerstein(AT)math.ucsb.edu), Mar 30 2007
   - Suppose psi=log(phi). We get the representation F(n)=(2/sqrt(5))*sinh(n*psi) if n is even; F(n)=(2/sqrt(5))*cosh(n*psi) if n is odd. There is a similar representation for Lucas numbers (A000032). Many Fibonacci formulas now easily follow from appropriate sinh and cosh formulas. For example: the de Moivre theorem (cosh(x)+sinh(x))^m=cosh(mx)+sinh(mx) produces L(n)^2+5F(n)^2=2L(2n) and L(n)F(n)=F(2n) (setting x=n*psi and m=2). - _Hieronymus Fischer_, Apr 18 2007
   - Inverse: floor(log_phi(sqrt(5)*Fib(n))+1/2)=n, for n>1. Also for n>0, floor(1/2*log_phi(5*Fib(n)*Fib(n+1)))=n. Extension valid for integer n, except n=0,-1: floor(1/2*sign(Fib(n)*Fib(n+1))*log_phi|5*Fib(n)*Fib(n+1)|)=n (where sign(x) = sign of x). - _Hieronymus Fischer_, May 02 2007
   - F(n+2) = The number of Khalimsky-continuous functions with a two-point codomain. - Shiva Samieinia (shiva(AT)math.su.se), Oct 04 2007
   - From Kauffman and Lopes, Proposition 8.2, p. 21: "The sequence of the determinants of the Fibonacci sequence of rational knots is the Fibonacci sequence (of numbers)." - _Jonathan Vos Post_, Oct 26 2007
   - This is a_1(n) in the Doroslovacki reference.
   - Let phi = (sqrt(5)+1)/2 = 1.6180339...; then phi^n = (1/phi)*a(n) + a(n+1). Example: phi^4 = 6.8541019... = (0.6180339...)*3 + 5. Also phi = 1/1 + 1/2 + 1/(2*5) + 1/(5*13) + 1/(13*34) + 1/(34*89) + ... - _Gary W. Adamson_, Dec 15 2007
   - The sequence of first differences, Fib(n+1)-Fib(n), is essentially the same sequence: 1, 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ... - _Colm Mulcahy_, Mar 03 2008
   - a(n)= the number of different ways to run up a staircase with n steps, taking steps of odd sizes where the order is relevant and there is no other restriction on the number or the size of each step taken. - _Mohammad K. Azarian_, May 21 2008
   - Equals row sums of triangle A144152. - _Gary W. Adamson_, Sep 12 2008
   - Except for the initial term, the numerator of the convergents to the recursion x = 1/(x+1). - _Cino Hilliard_, Sep 15 2008
   - F(n) is the number of possible binary sequences of length n that obey the sequential construction rule: if last symbol is 0, add the complement (1); else add 0 or 1. Here 0,1 are metasymbols for any 2-valued symbol set. This rule has obvious similarities to JFJ Laros's rule, but is based on addition rather than substitution and creates a tree rather than a single sequence. - _Ross Drewe_, Oct 05 2008
   - F(n) = Product_{k=1..(n-1)/2} (1 + 4*cos^2 k*Pi/n), where terms = roots to the Fibonacci product polynomials, A152063. - _Gary W. Adamson_, Nov 22 2008
   - Fp == 5^((p-1)/2) mod p, p = prime [Schroeder, p. 90]. - _Gary W. Adamson_ & _Alexander R. Povolotsky_, Feb 21 2009
   - (Ln)^2 - 5*(Fn)^2 = 4*(-1)^n. Example: 11^2 - 5*5 = -4. - _Gary W. Adamson_, Mar 11 2009
   - Output of Kasteleyn's formula for the number of perfect matchings of an m X n grid specializes to the Fibonacci sequence for m=2. - Sarah-Marie Belcastro (smbelcas(AT)toroidalsnark.net), Jul 04 2009
   - (Fib(n),Fib(n+4)) satisfies the Diophantine equation: X^2 + Y^2 - 7XY = 9*(-1)^n. - Mohamed Bouhamida (bhmd95(AT)yahoo.fr), Sep 06 2009
   - (Fib(n),Fib(n+2)) satisfies the Diophantine equation: X^2 + Y^2 - 3XY = (-1)^n. - Mohamed Bouhamida (bhmd95(AT)yahoo.fr), Sep 08 2009
   - a(n+2)=A083662(A131577(n)). - _Reinhard Zumkeller_, Sep 26 2009
   - Difference between of number of closed walks of length n+1 from a node on a pentagon and number of walks of length n+1 between two adjacent nodes on a pentagon. - _Henry Bottomley_, Feb 10 2010
   - F(n+1) = number of Motzkin paths of length n having exactly one weak ascent. A Motzkin path of length n is a lattice path from (0,0) to (n,0) consisting of U=(1,1), D=(1,-1) and H=(1,0) steps and never going below the x-axis. A weak ascent in a Motzkin path is a maximal sequence of consecutive U and H steps. Example: a(5)=5 because we have (HHHH), (HHU)D, (HUH)D, (UHH)D, and (UU)DD (the unique weak ascent is shown between parentheses; see A114690). - _Emeric Deutsch_, Mar 11 2010
   - (F(n-1) + F(n+1))^2 - 5F(n-2)*F(n+2) = 9*(-1)^n. - Mohamed Bouhamida (bhmd95(AT)yahoo.fr), Mar 31 2010
   - From the Pinter and Ziegler reference's abstract: authors "show that essentially the Fibonacci sequence is the unique binary recurrence which contains infinitely many three-term arithmetic progressions. A criterion for general linear recurrences having infinitely many three-term arithmetic progressions is also given." - _Jonathan Vos Post_, May 22 2010
   - F(n+1) = number of paths of length n starting at initial node on the path graph P_4. - _Johannes W. Meijer_, May 27 2010
   - F(k) = Number of cyclotomic polynomials in denominator of generating function for number of ways to place k nonattacking queens on an n X n board. - _Vaclav Kotesovec_, Jun 07 2010
   - As n-> inf., (a(n)/a(n-1) - a(n-1)/a(n)) tends to 1.0. Example: a(12)/a(11) - a(11)/a(12) = 144/89 - 89/144 = 0.99992197.... - _Gary W. Adamson_, Jul 16 2010
   - From _Hieronymus Fischer_, Oct 20 2010: (Start)
   - Fibonacci numbers are those numbers m such that m*phi is closer to an integer than k*phi for all k, 1<=k<m. More formally: a(0)=0, a(1)=1, a(2)=1, a(n+1)=minimal m>a(n) such that m*phi is closer to an integer than a(n)*phi.
   - For all numbers 1<=k<Fib(n), the inequality |k*phi-round(k*phi)| > |Fib(n)*phi-round(Fib(n)*phi)| holds.
   - Fib(n)*phi - round(Fib(n)*phi) = -((-phi)^(-n)), for n>1.
   - Fract(0.5+Fib(n)*phi) = 0.5 -(-phi)^(-n), for n>1.
   - Fract(Fib(n)*phi) = (1/2)*(1+(-1)^n)-(-phi)^(-n), n>1.
   - Inverse: n = -log_phi |0.5-fract(0.5+Fib(n)*phi)|.
   - (End)
   - F(A001177(n)*k) mod n = 0, for any integer k. - _Gary Detlefs_, Nov 27 2010
   - F(n+k)^2-F(n)^2 = F(k)*F(2n+k), for even k. - _Gary Detlefs_, Dec 04 2010
   - F(n+k)^2+F(n)^2 = F(k)*F(2n+k), for odd k. - _Gary Detlefs_, Dec 04 2010
   - F(n) = round(phi* F(n-1)) for n>1. - _Joseph P. Shoulak_, Jan 13 2012
   - For n > 0: a(n) = length of n-th row in Wythoff array A003603. - _Reinhard Zumkeller_, Jan 26 2012
   - From _Bridget Tenner_, Feb 22 2012: (Start)
   - The number of free permutations of [n].
   - The number of permutations of [n] for which s_k in supp(w) implies s_{k+-1} not in supp(w).
   - The number of permutations of [n] in which every decomposition into length(w) reflections is actually composed of simple reflections. (End)
   - The sequence F(n+1)^(1/n) is increasing. The sequence F(n+2)^(1/n) is decreasing. - _Thomas Ordowski_, Apr 19 2012
   - Two conjectures: For n > 1, F(n+2)^2 mod F(n+1)^2 = F(n)*F(n+1) - (-1)^n. For n > 0, (F(2n) + F(2n+2))^2 = F(4n+3) + sum_{k = 2..2n}F(2k). - _Alex Ratushnyak_, May 06 2012
   - From _Ravi Kumar Davala_, Jan 30 2014: (Start)
   - Proof of Ratushnyak's first conjecture: For n > 1, F(n+2)^2 - F(n)*F(n+1) + (-1)^n = 2F(n+1)^2.
   - Consider: F(n+2)^2 - F(n)*F(n+1) - 2F(n+1)^2
   -          = F(n+2)^2 - F(n+1)^2 - F(n+1)^2 - F(n)*F(n+1)
   -          =(F(n+2) + F(n+1))*(F(n+2) - F(n+1)) - F(n+1)*(F(n+1) + F(n))
   -          = F(n+3)*F(n) - F(n+1)*F(n+2) = -(-1)^n.
   - Proof of second conjecture: L(n) stands for Lucas number sequence from A000032.
   - Consider the fact that
   -     L(2n+1)^2 = L(4n+2) - 2
   -    (F(2n) + F(2n+2))^2 = F(4n+1) + F(4n+3) - 2
   -    (F(2n) + F(2n+2))^2 = sum{k = 2..2n, F(2k)} + F(4n+3).
   - (End)
   - The relationship: INVERT transform of (1,1,0,0,0,...) = (1, 2, 3, 5, 8,...), while the INVERT transform of (1,0,1,0,1,0,1,...) = (1, 1, 2, 3, 5, 8,...) is equivalent to: The numbers of compositions using parts 1 and 2 is equivalent to the numbers of compositions using parts == 1 mod 2 (i.e., the odd integers). Generally, the numbers of compositions using parts 1 and k is equivalent to the numbers of compositions of (n+1) using parts 1 mod k. Cf. A000930 for k = 3 and A003269 for k = 4. Example: for k = 2, n = 4 we have the compositions (22; 211, 121; 112; 1111) = 5; but using parts 1 and 3 we have for n = 5: (311, 131, 113, 11111, 5) = 5. - _Gary W. Adamson_, Jul 05 2012
   - The sequence F(n) is the binomial transformation of the alternating sequence (-1)^(n-1)*F(n), whereas the sequence F(n+1) is the binomial transformation of the alternating sequence (-1)^n*F(n-1). Both of these facts follow easily from the equalities a(n;1)=F(n+1) and b(n;1)=F(n) where a(n;d) and b(n;d) are so-called "delta-Fibonacci" numbers as defined in comments to A014445 (see also the papers of Witula et al.). - _Roman Witula_, Jul 24 2012
   - F(n) is the number of different (n-1)-digit binary numbers such that all substrings of length > 1 have at least one digit equal to 1. Example: for n = 5 there are 8 binary numbers with n - 1 = 4 digits (1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111), only the F(n) = 5 numbers 1010, 1011, 1101, 1110 and 1111 have the desired property. - _Hieronymus Fischer_, Nov 30 2012
   - For positive n, F(n+1) equals the determinant of the n X n tridiagonal matrix with 1's along the main diagonal, i's along the superdiagonal and along the subdiagonal where i = sqrt(-1). Example: Det([1,i,0,0; i,1,i,0; 0,i,1,i; 0,0,i,1]) = F(4+1) = 5. - _Philippe Deléham_, Feb 24 2013
   - For n>=1, number of compositions of n where there is a drop between every second pair of parts, starting with the first and second part; see example. Also, a(n+1) is the number of compositions where there is a drop between every second pair of parts, starting with the second and third part; see example. - _Joerg Arndt_, May 21 2013
   - Central terms of triangles in A162741 and A208245, n > 0. - _Reinhard Zumkeller_, Jul 28 2013
   - For n>=4, F(n-1) is the number of simple permutations in the geometric grid class given in A226433. - _Jay Pantone_, Sep 08 2013
   - a(n) are the pentagon (not pentagonal) numbers because the algebraic degree 2 number rho(5) = 2*cos(Pi/5) = phi (golden section), the length ratio diagonal/side in a pentagon, has minimal polynomial C(5,x) = x^2 - x - 1 (see A187360, n=5), hence rho(5)^n = a(n-1)*1 + a(n)*rho(5), n >= 0, in the power basis of the algebraic number field Q(rho(5)). One needs a(-1) = 1 here. See also the P. Steinbach reference under A049310. - _Wolfdieter Lang_, Oct 01 2013
   - A010056(a(n)) = 1. - _Reinhard Zumkeller_, Oct 10 2013
   - Define F(-n) to be F(n) for n odd and -F(n) for n even. Then for all n and k, F(n+2k)^2 - F(n)^2 = F(n+k)*( F(n+3k) - F(n-k) ). - _Charlie Marion_, Dec 20 2013
   - ( F(n), F(n+2k) ) satisfies the Diophantine equation: X^2 + Y^2 - L(2k)*X*Y = F(4k)^2*(-1)^n. This generalizes Bouhamida’s comments dated Sep 06 2009 and Sep 08 2009. - _Charlie Marion_, Jan 07 2014
   - For any prime p there is an infinite periodic subsequence within F(n) divisible by p, that begins at index n = 0 with value 0, and its first nonzero term at n = A001602(i), and period k = A001602(i). Also see A236479. - _Richard R. Forberg_, Jan 26 2014
   - Range of row n of the circular Pascal array of order 5. - _Shaun V. Ault_, May 30 2014 [orig. Kicey-Klimko 2011, and observations by Glen Whitehead; more general work found in Ault-Kicey 2014]
   - Nonnegative range of the quintic polynomial 2*y - y^5 + 2*x*y^4 + x^2*y^3 - 2*x^3*y^2 - x^4*y with x, y >= 0, see Jones 1975. - _Charles R Greathouse IV_, Jun 01 2014
   - The expression round(1/(F(k+1)/F(n) + F(k)/F(n+1))), for n > 0, yields a Fibonacci sequence with k-1 leading zeros (with rounding 0.5 to 0). - _Richard R. Forberg_, Aug 04 2014
   - Conjecture: For n > 0, F(n) is the number of all admissible residue classes for which specific finite subsequences of the Collatz 3n + 1 function consists of n+2 terms. This has been verified for 0 < n < 51. For details see Links. - _Mike Winkler_, Oct 03 2014
   - a(4)=3 and a(6)=8 are the only Fibonacci numbers that are of the form prime+1. - _Emmanuel Vantieghem_, Oct 02 2014
   - a(1)=1=a(2), a(3)=2 are the only Fibonacci numbers that are of the form prime-1. - _Emmanuel Vantieghem_, Jun 07 2015
   - Any consecutive pair (m, k) of the Fibonacci sequence a(n) illustrates a fair equivalence between m miles and k kilometers. For instance, 8 miles ~ 13 km; 13 miles ~ 21 km. -_Lekraj Beedassy_, Oct 06 2014
   - (n -> oo) lim (log F(n+1)/log F(n))^n = e. - _Thomas Ordowski_, Oct 06 2014
   - a(n+1) counts closed walks on K_2, containing one loop on the other vertex. Equivalently the (1,1)_entry of A^(n+1) where the adjacency matrix of digraph is A=(0,1; 1,1). - _David Neil McGrath_, Oct 29 2014
   - a(n-1) counts closed walks on the graph G(1-vertex;l-loop,2-loop). - _David Neil McGrath_, Nov 26 2014
   - From _Tom Copeland_, Nov 02 2014: (Start)
   - Let P(x) = x/(1+x) with comp. inverse Pinv(x) = x/(1-x) = -P[-x], and C(x)= [1-sqrt(1-4x)]/2, an o.g.f. for the shifted Catalan numbers A000108, with inverse Cinv(x) = x * (1-x).
   - Fin(x) = P[C(x)] = C(x)/[1 + C(x)] is an o.g.f. for the Fine numbers, A000957 with inverse Fin^(-1)(x) = Cinv[Pinv(x)] = Cinv[-P(-x)].
   - Mot(x) = C[P(x)] = C[-Pinv(-x)] gives an o.g.f. for shifted A005043, the Motzkin or Riordan numbers with comp. inverse Mot^(-1)(x) = Pinv[Cinv(x)] = (x - x^2) / (1 - x + x^2) (cf. A057078).
   - BTC(x) = C[Pinv(x)] gives A007317, a binomial transform of the Catalan numbers, with BTC^(-1)(x) = P[Cinv(x)].
   - Fib(x) = -Fin[Cinv(Cinv(-x))] = -P[Cinv(-x)] = x + 2 x^2 + 3 x^3 + 5 x^4 + ... = (x+x^2)/[1-x-x^2] is an o.g.f. for the shifted Fibonacci sequence A000045, so the comp. inverse is Fib^(-1)(x) = -C[Pinv(-x)] = -BTC(-x) and Fib(x) = -BTC^(-1)(-x).
   - Generalizing to P(x,t) = x /(1 + t*x) and Pinv(x,t) = x /(1 - t*x) = -P(-x,t) gives other relations to lattice paths, such as the o.g.f. for A091867, C[P[x,1-t]], and that for A104597, Pinv[Cinv(x),t+1].
   - (End)
   - In keeping with historical accounts (see the references by P. Singh and S. Kak), the generalized Fibonacci sequence a, b, a + b, a + 2b, 2a + 3b, 3a + 5b, ... can also be described as the Gopala-Hemachandra numbers H(n) = H(n-1) + H(n-2), with F(n) = H(n) for a = b = 1, and Lucas sequence L(n) = H(n) for a = 2, b = 1. - _Lekraj Beedassy_, Jan 11 2015
   - D. E. Knuth writes: "Before Fibonacci wrote his work, the sequence F_{n} had already been discussed by Indian scholars, who had long been interested in rhythmic patterns that are formed from one-beat and two-beat notes. The number of such rhythms having n beats altogether is F_{n+1}; therefore both Gopāla (before 1135) and Hemachandra (c. 1150) mentioned the numbers 1, 2, 3, 5, 8, 13, 21, ... explicitly." (TAOCP Vol. 1, 2nd ed.) - _Peter Luschny_, Jan 11 2015
   - F(n+1) equals the number of binary words of length n avoiding runs of zeroes of odd lengths. - _Milan Janjic_, Jan 28 2015
   - From _Russell Jay Hendel_, Apr 12 2015: (Start)
   - We prove Conjecture 1 of Rashid listed in the Formula section.
   - We use the following notation: F(n)=A000045(n), the Fibonacci numbers, and L(n) = A000032(n), the Lucas numbers. The fundamental Fibonacci-Lucas recursion asserts that G(n) = G(n-1)+ G(n-2), with "L" or "F" replacing "G".
   - We need the following prerequisites which we label (A), (B),(C), (D). The prerequisites are formulas in the Koshy book listed in the References section. (A) F(m-1)+F(m+1) = L(m) (Koshy, p. 97, #32), (B) L(2m)+2(-1)^m = L(m)^2 (Koshy p. 97, #41), (C) F(m+k)F(m-k) = (-1)^n F(k)^2 (Koshy, p. 113, #24, Tagiuri's identity), and (D) F(n)^2+F(n+1)^2 = F(2n+1) (Koshy, p. 97, #30).
   - We must also prove (E), L(n+2) F(n-1) = F(2n+1)+2(-1)^n. To prove (E), first note that by (A), proof of (E) is equivalent to proving that F(n+1)F(n-1) + F(n+3)F(n-1) = F(2n+1)+2(-1)^n. But by (C) with k=1, we have F(n+1)F(n-1) = F(n)^2 +(-1)^n. Applying (C) again with k=2 and m=n+1, we have F(n+3)F(n-1) = F(n+1)+(-1)^n. Adding these two applications of (C) together and using (D) we have, F(n+1)F(n-1) + F(n+3)F(n-1) = F(n)^2 + F(n+1)^2 + 2(-1)^n = F(2n+1)+2(-1)^n, completing the proof of (E).
   - We now prove Conjecture 1. By (A) and the Fibonacci-Lucas recursion, we have F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4) = [F(2n+1)+F(2n+3)] + [F(2n+2)+F(2n+4)] = L(2n+2)+L(2n+3)=L(2n+4). But then by (B), with m=2n+4, we have sqrt(L(2n+4)+2(-1)^n)) = L(n+2). Finally by (E), we have L(n+2) F(n-1)= F(2n+1)+2*(-1)^n. Dividing both sides by F(n-1), we have (F(2n+1)+2*(-1)^n)/F(n-1) = L(n+2) = sqrt(F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4)+2(-1)^n), as required.
   - (End)
   - In Fibonacci's Liber Abaci the rabbit problem appears in the translation of L. E. Sigler on pp. 404-405, and a remark [27] on p. 637. - _Wolfdieter Lang_, Apr 17 2015
   - a(n) counts partially ordered partitions of (n-1) into parts 1,2,3 where only the order of adjacent 1's and 2's are unimportant. (See example.) - _David Neil McGrath_, Jul 27 2015
   - F(n) divides F(nk). Proved by Marjorie Bicknell and Verner E Hoggatt Jr. - _Juhani Heino_, Aug 24 2015
   - F(n) is the number of UDU-equivalence classes of ballot paths of length n. Two ballot paths of length n with steps U = (1,1), D = (1,-1) are UDU-equivalent whenever the positions of UDU are the same in both paths. - _Kostas Manes_, Aug 25 2015
   - Cassini's identity F(2n+1) * F(2n+3) = F(2n+2)^2 + 1 is the basis for a geometrical paradox (or dissection fallacy) in A262342. - _Jonathan Sondow_, Oct 23 2015
   - For n >= 4, F(n) is the number of up-down words on alphabet {1,2,3} of length n-2. - _Ran Pan_, Nov 23 2015
   - F(n+2) is the number of terms in p(n), where p(n)/q(n) is the n-th convergent of the formal infinite continued fraction [a(0),a(1),...]; e.g., p(3) = a(0)a(1)a(2)a(3) + a(0)a(1) + a(0)a(3) + a(2)a(3) + 1 has F(5) terms. Also, F(n+1) is the number of terms in q(n). - _Clark Kimberling_, Dec 23 2015
   - F(n+1) (for n>=1) is the permanent of an n X n matrix M with M(i,j)=1 if |i-j|<=1 and 0 otherwise. - _Dmitry Efimov_, Jan 08 2016
   - A trapezoid has three sides of lengths in order F(n), F(n+2), F(n). For increasing n a very close approximation to the maximum area will have the fourth side equal to 2*F(n+1). For a trapezoid with lengths of sides in order F(n+2), F(n), F(n+2), the fourth side will be F(n+3). - _J. M. Bergot_, Mar 17 2016
   - (1) Join two triangles with lengths of sides L(n), F(n+3), L(n+2) and F(n+2), L(n+1), L(n+2) (where L(n)=A000032(n)) along the common side of length L(n+2) to create an irregular quadrilateral. Its area is approximately (5*F(2*n-1) - (F(2*n-7) - F(2*n-13))/5. (2) Join two triangles with lengths of sides L(n), F(n+2), F(n+3) and L(n+1), F(n+1, F(n+3) along the common side F(n+3) to form an irregular quadrilateral. Its area is approximately 4*F(2*n-1) - 2*(F(2*n-7) + F(2*n-18)). - _J. M. Bergot_, Apr 06 2016
   - From _Clark Kimberling_, Jun 13 2016: (Start)
   - Let T* be the infinite tree with root 0 generated by these rules: if p is in T*, then p+1 is in T* and x*p is in T*.
   - Let g(n) be the set of nodes in the n-th generation, so that g(0) = {0}, g(1) = {1}, g(2) = {2, x}, g(3) = {3, 2x, x+1, x^2}, etc.
   - Let T(r) be the tree obtained by substituting r for x.
   - If a positive integer N is not a square and r = sqrt(N), then the number of (not necessarily distinct) integers in g(n) is A000045(n), for n > = 1. See A274142. (End)
   - Consider the partitions of n, with all summands initially listed in nonincreasing order. Freeze all the 1's in place and then allow all the other summands to change their order, without displacing any of the 1's. The resulting number of arrangements is a(n+1). - _Gregory L. Simay_, Jun 14 2016

_Formulae_:
   - G.f.: x / (1 - x - x^2).
   - G.f.: Sum_{n>=0} x^n * Product_{k=1..n} (k + x)/(1 + k*x). - _Paul D. Hanna_, Oct 26 2013
   - F(n) = ((1+sqrt(5))^n-(1-sqrt(5))^n)/(2^n*sqrt(5)).
   - Alternatively, F(n) = ((1/2+sqrt(5)/2)^n-(1/2-sqrt(5)/2)^n)/sqrt(5).
   - F(n) = F(n-1) + F(n-2) = -(-1)^n F(-n).
   - F(n) = round(phi^n/sqrt(5)).
   - F(n+1) = Sum_{j=0..[n/2]} binomial(n-j, j).
   - This is a divisibility sequence; that is, if n divides m, then a(n) divides a(m). - _Michael Somos_, Apr 07 2012
   - E.g.f.: (2/sqrt(5))*exp(x/2)*sinh(sqrt(5)*x/2). - _Len Smiley_, Nov 30 2001
   - [0 1; 1 1]^n [0 1] = [F(n); F(n+1)]
   - x | F(n) ==> x | F(kn).
   - A sufficient condition for F(m) to be divisible by a prime p is (p - 1) divides m, if p == 1 or 4 (mod 5); (p + 1) divides m, if p == 2 or 3 (mod 5); or 5 divides m, if p = 5. (This is essentially Theorem 180 in Hardy and Wright.) - Fred W. Helenius (fredh(AT)ix.netcom.com), Jun 29 2001
   - a(n)=F(n) has the property: F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1). - _Miklos Kristof_, Nov 13 2003
   - _Kurmang. Aziz. Rashid_, Feb 21 2004, makes 4 conjectures and gives 3 theorems:
   - Conjecture 1: for n>=2 sqrt{F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4)+2*(-1)^n}={F(2n+1)+2*(-1)^n}/F(n-1). [For a proof see Comments section.]
   - Conjecture 2: for n>=0, {F(n+2)* F(n+3)}-{F(n+1)* F(n+4)}+ (-1)^n = 0.
   - Conjecture 3: for n>=0, F(2n+1)^3 - F(2n+1)*[(2*A^2)-1] - [A + A^3]=0, where A = {F(2n+1)+sqrt{5*F(2n+1)^2 +4}}/2.
   - Conjecture 4: for x>=5, if x is a Fibonacci number >= 5 then g*x*[{x+sqrt{5*(x^2) +- 4}}/2]*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[2x+{{3x+3*sqrt {5*(x^2) +- 4}}/2}]^2+[2x+{{x+sqrt{5*(x^2) +- 4}}/2}] +- x*[2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2 -x*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[x+{{x+sqrt{5*(x^2) +- 4}}/2}]* [2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2= 0, where g = {1 + sqrt(5)/2}.
   - Theorem 1: for n>=0, {F(n+3)^ 2 - F(n+1)^ 2}/F(n+2)={F(n+3)+ F(n+1)}.
   - Theorem 2: for n>=0, F(n+10) = 11*F(n+5) + F(n).
   - Theorem 3: for n>=6, F(n) = 4*F(n-3) + F(n-6).
   - Conjecture 2 of Rashid is actually a special case of the general law F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1) (take n <- n+1 and m <- -(n+4) in this law). - Harmel Nestra (harmel.nestra(AT)ut.ee), Apr 22 2005
   - Conjecture 2 of Rashid Kurmang simplified: F(n)*F(n+3) = F(n+1)*F(n+2)-(-1)^n. Follows from d'Ocagne's identity: m=n+2. - _Alex Ratushnyak_, May 06 2012
   - Conjecture: for all c such that 2-Phi <= c < 2*(2-Phi) we have F(n) = floor(Phi*a(n-1)+c) for n > 2. - _Gerald McGarvey_, Jul 21 2004
   - |2*Fib(n) - 9*Fib(n+1)| = 4*A000032(n) + A000032(n+1). - _Creighton Dement_, Aug 13 2004
   - For x > Phi, Sum_{n=0..inf} F(n)/x^n = x/(x^2 - x - 1) - _Gerald McGarvey_, Oct 27 2004
   - F(n+1) = exponent of the n-th term in the series f(x, 1) determined by the equation f(x, y) = xy + f(xy, x). - _Jonathan Sondow_, Dec 19 2004
   - a(n-1) = Sum_{k=0..n} (-1)^k*binomial(n-ceil(k/2), floor(k/2)). - _Benoit Cloitre_, May 05 2005
   - F(n+1) = Sum_{k=0..n} binomial((n+k)/2, (n-k)/2)(1+(-1)^(n-k))/2. - _Paul Barry_, Aug 28 2005
   - Fibonacci(n) = Product(1 + 4[cos(j*Pi/n)]^2, j=1..ceil(n/2)-1). [Bicknell and Hoggatt, pp. 47-48.] - _Emeric Deutsch_, Oct 15 2006
   - F(n) = 2^-(n-1)*Sum_{k=0..floor((n-1)/2)} binomial(n,2*k+1)*5^k. - _Hieronymus Fischer_, Feb 07 2006
   - a(n) = (b(n+1)+b(n-1))/n where {b(n)} is the sequence A001629. - _Sergio Falcon_, Nov 22 2006
   - F(n*m) = Sum_{k = 0..m} binomial(m,k)*F(n-1)^k*F(n)^(m-k)*F(m-k). The generating function of F(n*m) (n fixed, m = 0,1,2...) is G(x) = F(n)*x / ((1-F (n-1)*x)^2-F(n)*x*(1-F(n-1)*x)-( F(n)*x)^2). E.g., F(15) = 610 = F(5*3) = binomial(3,0)* F(4)^0*F(5)^3*F(3) + binomial(3,1)* F(4)^1*F(5)^2*F(2) + binomial(3,2)* F(4)^2*F(5)^1*F(1) + binomial(3,3)* F(4)^3*F(5)^0*F(0) = 1*1*125*2 + 3*3*25*1 + 3*9*5*1 + 1*27*1*0 = 250 + 225 + 135 + 0 = 610. - _Miklos Kristof_, Feb 12 2007
   - From _Miklos Kristof_, Mar 19 2007: (Start)
   -   Let L(n) = A000032(n) = Lucas numbers. Then:
   -   For a>=b and odd b, F(a+b)+F(a-b)=L(a)*F(b).
   -   For a>=b and even b, F(a+b)+F(a-b)=F(a)*L(b).
   -   For a>=b and odd b, F(a+b)-F(a-b)=F(a)*L(b).
   -   For a>=b and even b, F(a+b)-F(a-b)=L(a)*F(b).
   -   F(n+m)+(-1)^m*F(n-m)=F(n)*L(m);
   -   F(n+m)-(-1)^m*F(n-m)=L(n)*F(m);
   -   F(n+m+k)+(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =F(n)*L(m)*L(k);
   -   F(n+m+k)-(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =L(n)*L(m)*F(k);
   -   F(n+m+k)+(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =L(n)*F(m)*L(k);
   -   F(n+m+k)-(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =5*F(n)*F(m)*F(k). (End)
   - A corollary to Kristof 2007 is 2*F(a+b)=F(a)*L(b)+L(a)*F(b). - _Graeme McRae_, Apr 24 2014
   - For n>m, the sum of the 2m consecutive Fibonacci numbers F(n-m-1) thru F(n+m-2) is F(n)*L(m) if m is odd, and L(n)*F(m) if m is even (see the McRae link). - _Graeme McRae_, Apr 24 2014.
   - Fib(n) = b(n)+(p-1)*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital sum analog of the Fibonacci recurrence, defined by b(k)=ds_p(b(k-1))+ds_p(b(k-2)), b(0)=0, b(1)=1, ds_p=digital sum base p. Example for base p=10: Fib(n)=A010077(n)+9*Sum_{1<k<n} A059995(A010077(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - Fib(n) = b(n)+p*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital product analog of the Fibonacci recurrence, defined by b(k)=dp_p(b(k-1))+dp_p(b(k-2)), b(0)=0, b(1)=1, dp_p=digital product base p. Example for base p=10: Fib(n)=A074867(n)+10*Sum_{1<k<n} A059995(A074867(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - a(n) = denominator of continued fraction [1,1,1,...] (with n ones); e.g., 2/3 = continued fraction [1,1,1]; where barover[1] = [1,1,1...] = 0.6180339.... - _Gary W. Adamson_, Nov 29 2007
   - F(n + 3) = 2F(n + 2) - F(n), F(n + 4) = 3F(n + 2) - F(n), F(n + 8) = 7F(n + 4) - F(n), F(n + 12) = 18F(n + 6) - F(n). - _Paul Curtz_, Feb 01 2008
   - 1 = 1/(1*2) + 1/(1*3) + 1/(2*5) + 1/(3*8) + 1/(5*13) + ... = 1/2 + 1/3 + 1/10 + 1/24 + 1/65 + 1/168 + ...; where A059929 = (0, 2, 3, 10, 24, 65, 168,...). - _Gary W. Adamson_, Mar 16 2008
   - a(2^n) = prod{i=0}^{n-2}B(i) where B(i) is A001566. Example 3*7*47 = Fib(16). - _Kenneth J Ramsey_, Apr 23 2008
   - F(n) = (1/(n-1)!) * (n^(n-1) - (C(n-2,0) + 4*C(n-2,1) + 3*C(n-2,2))*n^(n-2) + (10*C(n-3,0) + 49*C(n-3,1) + 95*C(n-3,2) + 83*C(n-3,3) + 27*C(n-3,4))*n^(n-3) - (90*C(n-4,0) + 740*C(n-4,1) + 2415*C(n-4,2) + 4110*C(n-4,3) + 3890*C(n-4,4) + 1950*C(n-4,5) + 405*C(n-4,6))*n^(n-4) + ... ). - _André F. Labossière_, Nov 24 2004
   - a(n+1) = Sum_{k, 0<=k<=n} A109466(n,k)*(-1)^(n-k). -_Philippe Deléham_, Oct 26 2008
   - a(n) = Sum_{l_1=0..n+1} Sum_{l_2=0..n}...Sum_{l_i=0..n-i}... Sum_{l_n=0..1} delta(l_1,l_2,...,l_i,...,l_n), where delta(l_1,l_2,...,l_i,...,l_n) = 0 if any l_i + l_(i+1) >= 2 for i=1..n-1 and delta(l_1,l_2,...,l_i,...,l_n) = 1 otherwise. - _Thomas Wieder_, Feb 25 2009
   - a(n+1) = 2^n sqrt(Product_{k=1..n} cos(k Pi/(n+1))^2+1/4)) (Kasteleyn's formula specialized). - Sarah-Marie Belcastro (smbelcas(AT)toroidalsnark.net), Jul 04 2009
   - a(n+1) = Sum_{k=floor[n/2] mod 5} C(n,k) - Sum_{k=floor[(n+5)/2] mod 5} C(n,k) = A173125(n) - A173126(n) = |A054877(n)-A052964(n-1)|. - _Henry Bottomley_, Feb 10 2010
   - If p[i]=modp(i,2) and if A is Hessenberg matrix of order n defined by: A[i,j]=p[j-i+1], (i<=j), A[i,j]=-1, (i=j+1), and A[i,j]=0 otherwise. Then, for n>=1, a(n)=det A. - _Milan Janjic_, May 02 2010
   - Limit(F(k+n)/F(k), k = infinity) = (L(n) + F(n)*sqrt(5))/2 with the Lucas numbers L(n)= A000032(n). - _Johannes W. Meijer_, May 27 2010
   - For n>=1, F(n)=round(log_2(2^{phi*F(n-1)} + 2^{phi*F(n-2)})), where phi is the golden ratio. - _Vladimir Shevelev_, Jun 24 2010, Jun 27 2010
   - For n>=1, a(n+1)=ceil(phi*a(n)), if n is even and a(n+1)=floor(phi*a(n)), if n is odd (phi = golden ratio). - _Vladimir Shevelev_, Jul 01 2010
   - a(n) = 2*a(n-2) + a(n-3), n>2. - _Gary Detlefs_, Sep 08 2010
   - a(2^n) = Prod_{i=0..n-1} A000032(2^i). - _Vladimir Shevelev_, Nov 28 2010
   - a(n)^2 - a(n-1)^2 = a(n+1)*a(n-2), see A121646.
   - a(n) = sqrt((-1)^k*(a(n+k)^2 - a(k)*a(2n+k))), for any k. - _Gary Detlefs_, Dec 03 2010
   - F(2*n) = F(n+2)^2 - F(n+1)^2 - 2*F(n)^2. - _Richard R. Forberg_, Jun 04 2011
   - (-1)^(n+1) = F(n)^2 + F(n)*F(1+n) - F(1+n)^2.
   -   F(n) = -F(n+2)(-2 + (F(n+1))^4 + 2*(F(n+1)^3*F(n+2)) - (F(n+1)*F(n+2))^2 2*F(n+1)(F(n+2))^3 + (F(n+2))^4)- F(n+1). - _Artur Jasinski_, Nov 17 2011
   - F(n) = 1 + Sum_{x=1..n-2} F(x). - _Joseph P. Shoulak_, Feb 05 2012
   - F(n) = 4*F(n-2) - 2*F(n-3) - F(n-6). - _Gary Detlefs_, Apr 01 2012
   - F(n) = round(phi^(n+1)/(phi+2)). - _Thomas Ordowski_, Apr 20 2012
   - From _Sergei N. Gladkovskii_, Jun 03 2012: (Start)
   - G.f. A(x) = x/(1-x-x^2) = G(0)/sqrt(5) where G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*c^k/G(k+1))) and a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
   - Let E(x) be the e.g.f., i.e.,
   - E(x) = 1*x + 1/2*x^2 + 1/3*x^3 + 1/8*x^4 + 1/24*x^5 + 1/90*x^6 + 13/5040*x^7 + ...; then
   - E(x) = G(0)/sqrt(5); G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*(k+1)*c^k/G(k+1))), where a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
   - (End)
   - From _Hieronymus Fischer_, Nov 30 2012: (Start)
   - Fib(n) = 1 + Sum_{j_1=1..n-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} 1 + ... + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} ... Sum_{j_k=1..j_(k-1)-2} 1, where k = floor((n-1)/2).
   - Example: Fib(6) = 1 + Sum_{j=1..4} 1 + Sum_{j=1..4} Sum_{k=1..(j-2)} 1 + 0 = 1 + (1 + 1 + 1 + 1) + (1 + (1 + 1)) = 8.
   - Fib(n) = Sum_{j=0..k} S(j+1,n-2j), where k = floor((n-1)/2) and the S(j,n) are the n-th j-simplex sums: S(1,n) = 1 is the 1-simplex sum, S(2,n) = Sum_{k=1..n} S(1,k) = 1+1+...+1 = n is the 2-simplex sum, S(3,n) = Sum_{k=1..n} S(2,k) = 1+2+3+...+n is the 3-simplex sum (= triangular numbers = A000217), S(4,n) = Sum_{k=1..n} S(3,k) = 1+3+6+...+n(n+1)/2 is the 4-simplex sum (= tetrahedral numbers = A000292) and so on.
   - Since S(j,n) = binomial(n-2+j,j-1), the formula above equals the well-known binomial formula, essentially. (End)
   - G.f. A(x) = x / (1 - x / (1 - x / (1 + x))). - _Michael Somos_, Jan 04 2013
   - Sum_{n>=1} (-1)^(n-1)/(a(n)*a(n+1)) = 1/phi (phi=golden ratio). - _Vladimir Shevelev_, Feb 22 2013
   - From _Vladimir Shevelev_, Feb 24 2013: (Start)
   - (1) Expression a(n+1) via a(n): a(n+1) = (a(n) + sqrt(5*(a(n))^2 + 4*(-1)^n))/2;
   - (2) Sum_{k=1...n} (-1)^(k-1)/(a(k)*a(k+1)) = a(n)/a(n+1);
   - (3) a(n)/a(n+1) = 1/phi + r(n), where |r(n)| < 1/(a(n+1)*a(n+2)). (End)
   - F(n+1) = F(n)/2 + sqrt((-1)^n + 5*F(n)^2/4), n>=0. F(n+1) = U_n(i/2)/i^n, (U:= Chebyshef 2nd kind). - _Bill Gosper_, Mar 04 2013
   - G.f.: -Q(0) where Q(k) = 1 - (1+x)/(1 - x/(x - 1/Q(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Mar 06 2013
   - G.f.: x-1-1/x + 1/x/Q(0), where Q(k) = 1 - (k+1)*x/(1 - x/(x - (k+1)/Q(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Apr 23 2013
   - G.f.: x*G(0), where G(k)= 1 + x*(1+x)/(1 - x*(1+x)/(x*(1+x) + 1/G(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Jul 08 2013
   - G.f.: x^2 - 1 + 2*x^2/(W(0)-2), where W(k) = 1 + 1/(1 - x*(k + x)/( x*(k+1 + x) + 1/W(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Aug 28 2013
   - G.f.: Q(0) -1, where Q(k) = 1 + x^2 + (k+2)*x -x*(k+1 + x)/Q(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Oct 06 2013
   - Let b(n) = b(n-1) + b(n-2), with b(0) = 0, b(1) = phi. Then, for n>=2, F(n)= floor(b(n-1)) if n is even, F(n) = ceil(b(n-1)), if n is odd, with convergence. - _Richard R. Forberg_, Jan 19 2014
   - a(n) = Sum_{t1*g(1)+t2*g(2)+...+tn*g(n)=n} multinomial(t1+t2 +...+tn,t1,t2,...,tn), where g(k)=2*k-1. - _Mircea Merca_, Feb 27 2014
   - F(n) = round(sqrt(F(n-1)^2 + F(n)^2 + F(n+1)^2)/2), for n > 0. This rule appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Jul 27 2014
   - F(n) = round(2/(1/F(n) + 1/F(n+1) + 1/F(n+2)), for n > 0. This rule also appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Aug 03 2014
   - F(n) = round(1/(Sum_{j>=n+2} 1/F(j))). - _Richard R. Forberg_, Aug 14 2014
   - a(n) = hypergeometric([-n/2+1/2, -n/2+1], [-n+1], -4) for n>=2. - _Peter Luschny_, Sep 19 2014
   - F(n) = (L(n+1)^2 - L(n-1)^2)/(5*L(n)), where L(n) is A000032(n), with a similar inverse relationship. - _Richard R. Forberg_, Nov 17 2014
   - Consider the graph G[1-vertex;1-loop,2-loop] in comment above. Construct the power matrix array T(n,j)=[A^*j]*[S^*(j-1)] where A=(1,1,0,...) and S=(0,1,0,...)(A063524). [* is convolution operation] Define S^*0=I with I=(1,0,...). Then T(n,j) counts n-walks containing (j) loops and a(n-1) = Sum_{j=1...n} T(n,j). - _David Neil McGrath_, Nov 21 2014
   - Define F(-n) to be F(n) for n odd and -F(n) for n even. Then for all n and k, F(n) = F(k)*F(n-k+3) - F(k-1)*F(n-k+2) - F(k-2)*F(n-k) + (-1)^k*F(n-2k+2). - _Charlie Marion_, Dec 04 2014
   - F(n+k)^2 - L(k)*F(n)*F(n+k) + (-1)^k*F(n)^2 = (-1)^n*F(k)^2, if L(k) = A000032(k). - _Alexander Samokrutov_, Jul 20 2015
   - F(2*n) = F(n+1)^2 - F(n-1)^2, similar to Koshy (D) and Forberg 2011, but different. - _Hermann Stamm-Wilbrandt_, Aug 12 2015
   - F(n+1) = ceiling( (1/phi)*Sum_{k=0..n} F(k) ). - _Tom Edgar_, Sep 10 2015
   - a(n) = (L(n-3) + L(n+3))/10 where L(n)=A000032(n). - _J. M. Bergot_, Nov 25 2015
   - From _Bob Selcoe_, Mar 27 2016 (Start):
   - F(n) = (F(2n+k+1) - F(n+1)*F(n+k+1))/F(n+k), k>=0.
   - Thus when k=0: F(n) = sqrt(F(2n+1) - F(n+1)^2).
   - F(n) = cbrt(F(3n) - F(n+1)^3 + F(n-1)^3).
   - F(n+2k) = binomial transform of any subsequence starting with F(n). Example F(6)=8: 1*8 = F(6)=8; 1*8 + 1*13 = F(8)=21; 1*8 + 2*13 + 1*21 = F(10)=55; 1*8 + 3*13 + 3*21 + 1*34 = F(12)=144, etc. This formula applies to Fibonacci-type sequences with any two seed values for a(0) and a(1) (e.g., Lucas sequence A000032: a(0)=2, a(1)=1).
   - (End)
   - F(n) = L(k)*F(n-k) + (-1)^(k+1)*F(n-2k) for all k>=0, where L(k) = A000032(k). - _Anton Zakharov_, Aug 02 2016
   - From _Ilya Gutkovskiy_, Aug 03 2016: (Start)
   - a(n) = F_n(1), where F_n(x) are the Fibonacci polynomials.
   - Inverse binomial transform of A001906.
   - Number of zeros in substitution system {0 -> 11, 1 -> 1010} at step n from initial string "1" (1 -> 1010 -> 101011101011 -> ...) multiplied by 1/A000079(n). (End)

_Cross references_:
   - Cf. <a href="http://oeis.org/A039834">A039834</a> (signed Fibonacci numbers), <a href="http://oeis.org/A001690">A001690</a> (complement), <a href="http://oeis.org/A000213">A000213</a>, <a href="http://oeis.org/A000288">A000288</a>, <a href="http://oeis.org/A000322">A000322</a>, <a href="http://oeis.org/A000383">A000383</a>, <a href="http://oeis.org/A060455">A060455</a>, <a href="http://oeis.org/A030186">A030186</a>, <a href="http://oeis.org/A020695">A020695</a>, <a href="http://oeis.org/A020701">A020701</a>, <a href="http://oeis.org/A071679">A071679</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A100492">A100492</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A101399">A101399</a>, <a href="http://oeis.org/A101400">A101400</a>, <a href="http://oeis.org/A001611">A001611</a>, <a href="http://oeis.org/A000071">A000071</a>, <a href="http://oeis.org/A157725">A157725</a>, <a href="http://oeis.org/A001911">A001911</a>, <a href="http://oeis.org/A157726">A157726</a>, <a href="http://oeis.org/A006327">A006327</a>, <a href="http://oeis.org/A157727">A157727</a>, <a href="http://oeis.org/A157728">A157728</a>, <a href="http://oeis.org/A157729">A157729</a>, <a href="http://oeis.org/A167616">A167616</a>, <a href="http://oeis.org/A059929">A059929</a>, <a href="http://oeis.org/A144152">A144152</a>, <a href="http://oeis.org/A152063">A152063</a>, <a href="http://oeis.org/A114690">A114690</a>, <a href="http://oeis.org/A003893">A003893</a>, <a href="http://oeis.org/A000032">A000032</a>, <a href="http://oeis.org/A060441">A060441</a>, <a href="http://oeis.org/A000930">A000930</a>, <a href="http://oeis.org/A003269">A003269</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A057078">A057078</a>, <a href="http://oeis.org/A007317">A007317</a>, <a href="http://oeis.org/A091867">A091867</a>, <a href="http://oeis.org/A104597">A104597</a>, <a href="http://oeis.org/A249548">A249548</a>, <a href="http://oeis.org/A262342">A262342</a>.
   - First row of arrays <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A234357">A234357</a>. Second row of arrays <a href="http://oeis.org/A099390">A099390</a>, <a href="http://oeis.org/A048887">A048887</a>, and <a href="http://oeis.org/A092921">A092921</a> (k-generalized Fibonacci numbers).
   - a(n) = <a href="http://oeis.org/A094718">A094718</a>(4, n). a(n) = <a href="http://oeis.org/A101220">A101220</a>(0, j, n).
   - a(n) = <a href="http://oeis.org/A090888">A090888</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(1, n-1) = <a href="http://oeis.org/A109754">A109754</a>(0, n) = <a href="http://oeis.org/A109754">A109754</a>(1, n-1), for n > 0.
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.
   - Boustrophedon transforms: <a href="http://oeis.org/A000738">A000738</a>, <a href="http://oeis.org/A000744">A000744</a>.
   - Powers: <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A105317">A105317</a>, <a href="http://oeis.org/A254719">A254719</a>.
   - Numbers of prime factors: <a href="http://oeis.org/A022307">A022307</a> and <a href="http://oeis.org/A038575">A038575</a>.

_Links_:
   - N. J. A. Sloane, <a href="/A000045/b000045.txt">The first 2000 Fibonacci numbers: Table of n, F(n) for n = 0..2000</a>
   - Amazing Mathematical Object Factory, <a href="http://www.schoolnet.ca/vp-pv/amof/e_fiboI.htm">Information on the Fibonacci sequences</a>
   - Matt Anderson, Jeffrey Frazier and Kris Popendorf, <a href="http://library.thinkquest.org/27890/theSeries.html">The Fibonacci series: the section index</a> [broken link]
   - P. G. Anderson, <a href="http://www.cs.rit.edu/~pga/Fibo/fact_sheet.html">Fibonacci Facts</a>
   - Joerg Arndt, <a href="http://www.jjj.de/fxt/#fxtbook">Matters Computational (The Fxtbook)</a>
   - J.-L. Baril, <a href="http://www.combinatorics.org/Volume_18/PDF/v18i1p178.pdf">Classical sequences revisited with permutations avoiding dotted pattern</a>, Electronic Journal of Combinatorics, 18 (2011), #P178.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Barry/barry84.html">A Catalan Transform and Related Transformations on Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.5.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.
   - A. T. Benjamin, A. K. Eustis, M. A. Shattuck, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Shattuck/shattuck20.html">Compression Theorems for Periodic Tilings and Consequences</a>, JIS 12 (2009) 09.6.3
   - E. R. Berlekamp, <a href="/A257113/a257113.pdf">A contribution to mathematical psychometrics</a>, Unpublished Bell Labs Memorandum, Feb 08 1968 [Annotated scanned copy]
   - Marjorie Bicknell and Verner E Hoggatt Jr, <a href="http://www.fq.math.ca/Books/Primer/bicknell6.pdf">To Prove: F(n) Divides F(nk)</a>, A Primer for the Fibonacci Numbers: IX (1973)
   - J. Bodeen, S. Butler, T. Kim, X. Sun, S. Wang, <a href="http://www.combinatorics.org/ojs/index.php/eljc/article/view/v21i1p7">Tiling a strip with triangles</a>, El. J. Combinat. 21 (1) (2014) P1.7
   - H. Bottomley and N. J. A. Sloane, <a href="/A000045/a000045.html">Illustration of initial terms: the Fibonacci tree</a>
   - Brantacan, <a href="http://www.branta.connectfree.co.uk/fibonacci.htm">Fibonacci Numbers</a> [broken link]
   - J. Britton & B. V. Eeckhout, <a href="http://ccins.camosun.bc.ca/~jbritton/fibonacci/jbfibapplet.htm">Fibonacci Interactive</a> [broken link]
   - S. Brlek, E. Duchi, E. Pergola and S. Rinaldi, <a href="http://dx.doi.org/10.1016/j.disc.2004.07.019">On the equivalence problem for succession rules</a>, Discr. Math., 298 (2005), 142-154.
   - N. D. Cahill, J. R. D'Errico, J. P. Spence, <a href="http://www.fq.math.ca/41-1.html">Complex factorizations of the Fibonacci and Lucas numbers</a>, Fib. Quart. 41 (2003) 13.
   - N. D. Cahill and D. A. Narayan, <a href="http://www.fq.math.ca/Papers1/42-3/quartcahill03_2004.pdf">Fibonacci and Lucas Numbers as Tridiagonal Matrix Determinants</a>, Fibonacci Quarterly, 42(3):216-221, 2004.
   - C. K. Caldwell, The Prime Glossary, <a href="http://primes.utm.edu/glossary/page.php/FibonacciNumber.html">Fibonacci number</a>
   - P. J. Cameron, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/groups.html">Sequences realized by oligomorphic permutation groups</a>, J. Integ. Seqs. Vol. 3 (2000), #00.1.5.
   - Hongwei Chen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Chen/chen78.html">Evaluations of Some Variant Euler Sums</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.3.
   - J. H. Conway and N. J. A. Sloane, <a href="/A019586/a019586.pdf">Notes on the Para-Fibonacci and related sequences</a>
   - J. H. Conway, Allan Wechsler, Marc LeBrun, Dan Hoey, N. J. A. Sloane, <a href="/A269725/a269725.txt">On Kimberling Sums and Para-Fibonacci Sequences</a>, Correspondence and Postings to Math-Fun Mailing List, Nov 1996 to Jan 1997
   - E. S. Croot, <a href="http://www.math.gatech.edu/~ecroot/recurrence_notes2.pdf">Notes on Linear Recurrence Sequences</a>
   - Paul Cubre, <a href="http://wakespace.lib.wfu.edu/bitstream/handle/10339/37313/Cubre_wfu_0248M_10299.pdf">The Z-densities of the Fibonacci sequence</a>, M. A. Thesis, Wake Forest University, May 2012;
   - C. Dement, <a href="http://mathforum.org/discuss/sci.math/t/622432">Posting to Math Forum</a> [broken link].
   - R. M. Dickau, <a href="http://mathforum.org/advanced/robertd/fibboard.html">Fibonacci numbers</a>
   - R. Doroslovacki, <a href="http://www.emis.de/journals/MV/9434/mv943407.ps">Binary sequences without 011...110 (k-1 1's) for fixed k</a>, Mat. Vesnik 46 (1994), no. 3-4, 93-98.
   - A.-S. Elsenhans and J. Jahnel, <a href="http://www.uni-math.gwdg.de/tschinkel/gauss/Fibon.pdf">The Fibonacci sequence modulo p^2 - an investigation by computer for p < 10^14</a>.
   - Nathaniel D. Emerson, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Emerson/emerson6.html">A Family of Meta-Fibonacci Sequences Defined by Variable-Order Recursions</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.8.
   - Reinhardt Euler, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Euler/euler1.html">The Fibonacci Number of a Grid Graph and a New Class of Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.2.6.
   - G. Everest, A. J. van der Poorten, Y. Puri and T. Ward, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL5/Ward/ward2.html">Integer Sequences and Periodic Points</a>, Journal of Integer Sequences, Vol. 5 (2002), Article 02.2.3
   - G. Everest, A. van der Poorten, I. Shparlinski and T. Ward, Recurrence Sequences, Amer. Math. Soc., 2003; see esp. p. 255. <a href="http://www.ams.org/mathscinet-getitem?mr=1990179">MR1990179</a>
   - Philipp Fahr and Claus Michael Ringel, <a href="http://www.mathematik.uni-bielefeld.de/~ringel/opus/fr-zwei.pdf">Categorification of the Fibonacci Numbers Using Representations of Quivers</a>
   - John Farrier, <a href="http://uploads.neatorama.com/wp-content/uploads/2010/09/1nT6a-500x296.jpg">Fibonacci Pigeons</a> [From Sarah Spolaor, Sep 30 2010]
   - Helaman and Claire Ferguson, <a href="http://www.ams.org/notices/201007/rtx100700840p.pdf">Celebrating Mathematics in Stone and Bronze</a>, Notices of the American Mathematical Society, 57 (2010), 840-850. See page 844
   - Emmanuel Ferrand, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL10/Ferrand/ferrand8.html">Deformations of the Taylor Formula</a>, Journal of Integer Sequences, Vol. 10 (2007), Article 07.1.7.
   - D. Foata and G.-N. Han, <a href="http://www-irma.u-strasbg.fr/~foata/paper/pub71.html">Nombres de Fibonacci et polynomes orthogonaux</a>,
   - I. Galkin, <a href="http://ulcar.uml.edu/~iag/CS/Fibonacci.html">"Fibonacci Numbers Spelled Out"</a>
   - Dale Gerdemann, <a href="http://www.youtube.com/watch?v=YpTSfkqS7zg">Video of Fibonacci tree</a>
   - Dale Gerdemann, <a href="http://bit.ly/lw7pP2">Video of Fibonacci tree(s)</a> (another version)
   - Dale Gerdemann, <a href="http://www.youtube.com/watch?v=AvRdzZMcUrI">Golden Ratio Base Contains Zeckendorf and Negative Indexed Bunder Forms</a>
   - C. J. Glasby, S. P. Glasby, F. Pleijel, <a href="http://dx.doi.org/10.1098/rspb.2008.0418">Worms by number</a>, Proc. Roy. Soc. B, Proc. Biol. Sci. 275 (1647) (2008) 2071-2076.
   - L. Goldsmith, <a href="http://people.bath.ac.uk/ma2lag/fibonaccinumbers.html">The Fibonacci Numbers</a> [broken link]
   - Hank Green, <a href="https://www.youtube.com/watch?v=wTlw7fNcO-0">The Fibonacci Sequence: Nature's Code</a> (2012).
   - M. Griffiths, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL14/Griffiths/griffiths16.html">A Restricted Random Walk defined via a Fibonacci Process</a>, Journal of Integer Sequences, Vol. 14 (2011), #11.5.4.
   - Nancy S. S. Gu, Nelson Y. Li, and Toufik Mansour, <a href="http://dx.doi.org/10.1016/j.disc.2007.04.007">2-Binary trees: bijections and related issues</a>, Discr. Math., 308 (2008), 1209-1221.
   - Guo-Niu Han, <a href="http://www-irma.u-strasbg.fr/~guoniu/papers/p77puzzle.pdf">Enumeration of Standard Puzzles</a> [broken link]
   - Guo-Niu Han, <a href="/A196265/a196265.pdf">Enumeration of Standard Puzzles</a> [Cached copy]
   - S. Happersett, <a href="http://dx.doi.org/10.1080/17513470701210626">Mathematical meditations</a>
   - Brady Haran and Colm Mulcahy, <a href="https://www.youtube.com/watch?v=4izjrtR8Ozg">Little Fibs - Numberphile</a> (2016)
   - A. P. Hillman & G. L. Alexanderson, Algebra Through Problem Solving, Chapter 2 pp. 11-16, <a href="http://education.lanl.gov/RESOURCES/ATPS/CHPTR02/P011.HTM">The Fibonacci and Lucas Numbers</a> [broken link]
   - A. M. Hinz, S. Klavžar, U. Milutinović, C. Petr, <a href="http://dx.doi.org/10.1007/978-3-0348-0237-6">The Tower of Hanoi - Myths and Maths</a>, Birkhäuser 2013. See page 12. <a href="http://tohbook.info">Book's website</a>
   - V. E. Hoggatt and C. T. Long, <a href="http://www.fq.math.ca/Scanned/12-2/hoggatt1.pdf">Divisibility Properties of Generalized Fibonacci Polynomials</a>, Fibonacci Quarterly, 12:113-120, 1974.
   - C. W. Huegy and D. B. West, <a href="http://dx.doi.org/10.1016/S0012-365X(01)00239-4">A Fibonacci tiling of the plane</a>, Discrete Math., 249 (2002), 111-116.
   - INRIA Algorithms Project, <a href="http://algo.inria.fr/ecs/ecs?searchType=1&amp;service=Search&amp;searchTerms=9">Encyclopedia of Combinatorial Structures 9</a> [broken link]
   - Tina Hill Janzen, <a href="http://www.youtube.com/watch?v=2nAycC7sGVI">Fibonacci sequence / Golden scale</a> [broken link]
   - Q.-H. Hou, Z.-W. Sun and H.-M. Wen, <a href="http://arxiv.org/abs/1208.3903">On monotonicity of some combinatorial sequences</a>, arXiv:1208.3903 [math.CO], 2012-2014.
   - James P. Jones, <a href="http://www.fq.math.ca/Scanned/13-1/jones.pdf">Diophantine representation of the Fibonacci numbers</a>, Fibonacci Quarterly 13:1 (1975), pp. 84-88.
   - R. Jovonovic, <a href="http://milan.milanovic.org/math/english/function/function.html">Fibonacci Function Calculator</a> [broken link]
   - R. Jovonovic, <a href="http://milan.milanovic.org/math/english/pdf/Fibonacci.pdf">The relations between the Fibonacci and the Lucas numbers</a> [broken link]
   - R. Jovanovic, <a href="http://milan.milanovic.org/math/Math.php?akcija=SviFibo">First 70 Fibonacci numbers</a> [broken link]
   - S. Kak, <a href="http://arXiv.org/abs/physics/0411195">The Golden Mean and the Physics of Aesthetics</a>, arXiv:physics/0411195 [physics.hist-ph], 2004.
   - Louis H. Kauffman and Pedro Lopes, <a href="http://arXiv.org/abs/0710.3765">Graded forests and rational knots</a>, arXiv:0710.3765 [math.GT], 2007-2009.
   - Blair Kelly, <a href="http://mersennus.net/fibonacci//">Fibonacci and Lucas factorizations</a>
   - Tanya Khovanova, <a href="http://www.tanyakhovanova.com/RecursiveSequences/RecursiveSequences.html">Recursive Sequences</a>
   - C. Kimberling, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL6/Kimberling/kimberling24.html">Matrix Transformations of Integer Sequences</a>, J. Integer Seqs., Vol. 6, 2003.
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/">Fibonacci numbers and the golden section</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibmaths.html">Mathematics of the Fibonacci Series</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibtable.html">Fibonacci numbers with tables of F(0)-F(500).</a>
   - A. Krowne, PlanetMath.org, <a href="http://planetmath.org/encyclopedia/FibonacciNumber.html">Fibonacci sequence</a> [broken link]
   - Hendrik Lenstra, <a href="http://math.berkeley.edu/~hwl/papers/fibo.pdf">Profinite Fibonacci Numbers</a>
   - M. A. Lerma, <a href="http://www.math.northwestern.edu/~mlerma/problem_solving/results/recurrences.pdf">Recurrence Relations</a>
   - D. Litchfield, D. Goldenheim and C. H. Dietrich, <a href="http://scientium.com/diagon_alley/archival/segments/euclid1.htm">Euclid, Fibonacci and Sketchpad</a>, Math. Teacher, 90 (1997). [broken link]
   - F. Luca, V. J. M. Huguet, F. Nicolae, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Mejia/luca31.html">On the Euler Function of Fibonacci Numbers</a>, JIS 12 (2009) 09.6.6
   - B. Malesevic, <a href="http://matematika.etf.bg.ac.yu/publikacije/pub/P09(98)/P09_06.ZIP">Some combinatorial aspects of differential operation composition on the space R^n </a> [broken link].
   - Charles P. McKeague, <a href="http://www.youtube.com/watch?v=_NmSEEhtc1U">Fibonacci numbers from MathTV</a>
   - Graeme McRae, <a href="http://oeis.org/wiki/User:Graeme_McRae/Sum_of_2m_Consecutive_Fibonacci_Numbers">Sum of 2m consecutive Fibonacci numbers</a>
   - D. Merlini, R. Sprugnoli and M. C. Verri, <a href="http://www.dsi.unifi.it/~merlini/tiling.ps">Strip tiling and regular grammars</a>, Theoret. Computer Sci. 242, 1-2 (2000) 109-124. [broken link]
   - D. Merrill, <a href="http://pw1.netcom.com/~merrills/fibphi.html">The Fib-Phi Link Page</a> [broken link]
   - Jean-Christophe Michel, <a href="http://framy.free.fr/fibonacci%20dans%20mandelbrot.htm">Le nombre d'or dans l'ensemble de Mandelbrot</a> (in French, 'The golden number in the Mandelbrot set')
   - Kerry Mitchell, <a href="http://kerrymitchellart.com/articles/Spirolateral-Type_Images_from_Integer_Sequences.pdf">Spirolateral-Type Images from Integer Sequences</a>, 2013
   - H. Mishima, Factorizations of Fibonacci numbers <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha108.htm">n=1..100</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha109.htm">n=101..200</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha110.htm">n=201..300</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha111.htm">n=301..400</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha112.htm">n=401..480</a>
   - P. Moree, <a href="http://arXiv.org/abs/math.CO/0311205">Convoluted convolved Fibonacci numbers</a>, arXiv:math/0311205 [math.CO], 2003.
   - R. Mullen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Mullen/mullen2.html">On Determining Paint by Numbers Puzzles with Nonunique Solutions</a>, JIS 12 (2009) 09.6.5
   - Newton Institute, <a href="http://www.newton.cam.ac.uk/wmy2kposters/january">Posters in the London Underground</a>
   - Tony D. Noe and Jonathan Vos Post, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Noe/noe5.html">Primes in Fibonacci n-step and Lucas n-step Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.4.
   - J. J. O'Connor and E. F. Robertson, Mac Tutor History of Mathematics, <a href="http://www-history.mcs.st-andrews.ac.uk/Biographies/Hemchandra.html">Archarya Hemachandra</a>
   - Arzu Özkoç, <a href="http://link.springer.com/article/10.1186/s13662-015-0486-7/fulltext.html">Some algebraic identities on quadra Fibona-Pell integer sequence</a>, Advances in Difference Equations, 2015, 2015:148.
   - Ram Krishna Pandey, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Pandey/pandey7.html">On Some Magnified Fibonacci Numbers Modulo a Lucas Number</a>, Journal of Integer Sequences, Vol. 16 (2013), #13.1.7.
   - J. Patterson, <a href="http://www.bath.ac.uk/~ma1jmp/link.html">The Fibonacci Sequence</a>. [broken link]
   - Ed Pegg, Jr., <a href="http://www.mathpuzzle.com/MAA/07-Sequence%20Pictures/mathgames_12_08_03.html">Sequence Pictures</a>, Math Games column, Dec 08 2003.
   - Ed Pegg, Jr., <a href="/A000043/a000043_2.pdf">Sequence Pictures</a>, Math Games column, Dec 08 2003 [Cached copy, with permission (pdf only)]
   - T. K. Petersen and B. E. Tenner, <a href="http://arxiv.org/abs/1202.4765">The depth of a permutation</a>, arXiv:1202.4765v1 [math.CO], 2012-2014.
   - Ivars Peterson, <a href="http://www.sciencenews.org/articles/20060603/mathtrek.asp">Fibonacci's Missing Flowers</a>.
   - Akos Pinter and Volker Ziegler, <a href="http://arxiv.org/abs/1005.3624">On Arithmetic Progressions in Recurrences - A new characterization of the Fibonacci sequence</a>, arXiv:1005.3624 [math.NT], 2010.
   - Simon Plouffe, Project Gutenberg, <a href="http://ibiblio.org/pub/docs/books/gutenberg/etext01/fbncc10.txt">The First 1001 Fibonacci Numbers</a> [broken link]
   - Project Nayuki, <a href="http://www.nayuki.io/page/fast-fibonacci-algorithms">Fast Fibonacci algorithms</a> (fast doubling is faster than matrix multiplication).
   - S. Rabinowitz, <a href="http://www.mathpropress.com/stan/bibliography/algorithmicFib.pdf">Algorithmic Manipulation of Fibonacci Identities</a> (1996).
   - Arulalan Rajan, R. Vittal Rao, Ashok Rao and H. S. Jamadagni, <a href="http://arxiv.org/abs/1205.5398">Fibonacci Sequence, Recurrence Relations, Discrete Probability Distributions and Linear Convolution</a>, arXiv preprint arXiv:1205.5398 [math.PR], 2012. - From _N. J. A. Sloane_, Oct 23 2012
   - Marc Renault, <a href="http://www.math.temple.edu/~renault/fibonacci/thesis.html">Properties of the Fibonacci sequence under various moduli</a>, MSc Thesis, Wake Forest U, 1996.
   - N. Renton, <a href="http://www.users.bigpond.net.au/renton/903.htm">The fibonacci Series</a>
   - B. Rittaud, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL10/Rittaud2/rittaud11.pdf">On the Average Growth of Random Fibonacci Sequences</a>, Journal of Integer Sequences, 10 (2007), Article 07.2.4.
   - Rosetta Code, <a href="http://rosettacode.org/wiki/Fibonacci_sequence">A collection of codes to compute fibonacci numbers with different computer languages</a>
   - E. S. Rowland, <a href="http://people.hofstra.edu/Eric_Rowland/investigations/fibonacci.html">Fibonacci Sequence Calculator up to n=1474</a>
   - Michelle Rudolph-Lilith, <a href="http://arxiv.org/abs/1508.07894">On the Product Representation of Number Sequences, with Application to the Fibonacci Family</a>, arXiv preprint arXiv:1508.07894, 2015
   - Shiva Samieinia, <a href="http://www.math.su.se/reports/2007/6/">Digital straight line segments and curves</a>. Licentiate Thesis. Stockholm University, Department of Mathematics, Report 2007:6.
   - A. Sapounakis, I. Tasoulas and P. Tsikouras, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Tsikouras/tsikouras67.html">On the Dominance Partial Ordering of Dyck Paths</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.5.
   - D. Schweizer, <a href="http://math.holycross.edu/~davids/fibonacci/fibonacci.html">First 500 Fibonacci Numbers in blocks of 100.</a> [broken link]
   - Mark A. Shattuck, <a href="http://www.emis.de/journals/INTEGERS/papers/j5/j5.Abstract.html">Tiling proofs of some formulas for the Pell numbers of odd index</a>, Integers, 9 (2009), 53-64.
   - Mark A. Shattuck and Carl G. Wagner, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Shattuck/shattuck56.html">Periodicity and Parity Theorems for a Statistic on r-Mino Arrangements</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.3.6.
   - S. Silvia, <a href="http://arttech.about.com/library/weekly/aa060900a_fibonacci_sequence.htm">Fibonacci sequence</a> [broken link]
   - Parmanand Singh, <a href="http://dx.doi.org/10.1016/0315-0860(85)90021-7">The so-called Fibonacci numbers in ancient and medieval India</a>, Historia Mathematica, Volume 12 (3), 1985, 229-244.
   - Jaap Spies, <a href="http://www.jaapspies.nl/oeis/a000045.sage">Sage program for computing A000045</a>
   - Michael Z. Spivey and Laura L. Steil, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Spivey/spivey7.html">The k-Binomial Transforms and the Hankel Transform</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.1.
   - Z.-H. Sun, <a href="http://202.195.112.2/xsjl/szh/ConFn.pdf">Congruences For Fibonacci Numbers</a>
   - Roberto Tauraso, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Tauraso/tauraso3.html">A New Domino Tiling Sequence</a>, Journal of Integer Sequences, Vol. 7 (2004), Article 04.2.3.
   - Thesaurus.Maths.org, <a href="http://thesaurus.maths.org/dictionary/map/word/3788">Fibonacci sequence</a> [broken link]
   - K. Tognetti, <a href="/A000045/a000045.pdf">Letter to N. J. A. Sloane (with attachments), May 25 1994</a>
   - K. Tognetti, <a href="/A000045/a000045_2.pdf">The Search for the Golden Sequence</a>, Draft Manuscript, May 25 1994.
   - K. Tognetti, <a href="http://www.austms.org.au/Modules/Fib">Fibonacci-His Rabbits and His Numbers and Kepler</a>
   - Tony van Ravenstein, <a href="/A000045/a000045_1.pdf">The three gap theorem (Steinhaus conjecture)</a>, Journal of the Australian Mathematical Society (Series A) 45.03 (1988): 360-370. [Annotated scanned copy]
   - C. Vila, <a href="http://www.boingboing.net/2010/03/22/dreamlike-animation.html">Nature by numbers</a> (animation).
   - Christobal Vila, <a href="http://wimp.com/naturenumbers/">Nature Numbers</a> (Video related to Fibonacci numbers)
   - N. N. Vorob'ev, <a href="http://eom.springer.de/F/f040020.htm">Fibonacci numbers</a>, Springer's Encyclopaedia of Mathematics.
   - Carl G. Wagner, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Wagner/wagner3.html">Partition Statistics and q-Bell Numbers (q = -1)</a>, J. Integer Seqs., Vol. 7, 2004.
   - Robert Walker, <a href="http://www.youtube.com/watch?v=Wx4ZfuMl-FI&amp;NR=1">Inharmonic "Golden Rhythmicon" - Fibonacci Sequence in Pairs Approaching Golden Ratio - With Bounce</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/FibonacciNumber.html">Fibonacci Number</a>, <a href="http://mathworld.wolfram.com/Double-FreeSet.html">Double-Free Set</a>, <a href="http://mathworld.wolfram.com/Fibonaccin-StepNumber.html">Fibonacci n-Step Number</a>, <a href="http://mathworld.wolfram.com/ResistorNetwork.html">Resistor Network</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/HosoyaIndex.html">Hosoya Index</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/PathGraph.html">Path Graph</a>
   - Wikipedia, <a href="http://www.wikipedia.org/wiki/Fibonacci_number">Fibonacci number</a>
   - Wikipedia, <a href="https://en.wikipedia.org/wiki/Cassini_and_Catalan_identities">Cassini and Catalan identities</a>
   - Willem's Fibonacci site, <a href="http://home.zonnet.nl/LeonardEuler/fiboe.htm">Fibonacci</a>
   - Mike Winkler, <a href="http://arxiv.org/abs/1412.0519">On the structure and the behaviour of Collatz 3n + 1 sequences - Finite subsequences and the role of the Fibonacci sequence</a>, arXiv:1412.0519 [math.GM], 2014.
   - Roman Witula, Damian Slota and Edyta Hetmaniok, <a href="http://ami.ektf.hu/uploads/papers/finalpdf/AMI_41_from255to263.pdf">Bridges between different known integer sequences</a>, Annales Mathematicae et Informaticae, 41 (2013) pp. 255-263.
   - Aimei Yu and Xuezheng Lv, <a href="http://dx.doi.org/10.1007/s10910-006-9088-7">The Merrifield-Simmons indices and Hosoya indices of trees with k pendant vertices</a>, J. Math. Chem., Vol. 41 (2007), pp. 33-43. See page 35.
   - Tianping Zhang and Yuankui Ma, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Zhang/zhang56.html">On Generalized Fibonacci Polynomials and Bernoulli Numbers</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.5.3.
   - <a href="/index/Cor#core">Index entries for "core" sequences</a>
   - <a href="/index/Di#divseq">Index to divisibility sequences</a>
   - <a href="/index/Par#partN">Index entries for related partition-counting sequences</a>
   - <a href="/index/Rec#order_02">Index entries for linear recurrences with constant coefficients</a>, signature (1,1).
   - <a href="/index/Tu#2wis">Index entries for two-way infinite sequences</a>

_References_:
   - Abrate, Marco; Barbero, Stefano; Cerruti, Umberto; Murru, Nadir. Colored compositions, Invert operator and elegant compositions with the "black tie". Discrete Math. 335 (2014), 1-7. MR3248794
   - Andrews, George E. Fibonacci numbers and the Rogers-Ramanujan identities. Fibonacci Quart. 42 (2004), no. 1, 3-19. MR2060558(2005b:11161)
   - S. V. Ault and C. Kicey, Counting paths in corridors using circular Pascal arrays, Discrete Mathematics (2014).
   - Mohammad K. Azarian, The Generating Function for the Fibonacci Sequence, Missouri Journal of Mathematical Sciences, Vol. 2, No. 2, Spring 1990, pp. 78-79. Zentralblatt MATH, Zbl 1097.11516.
   - Mohammad K. Azarian, A Generalization of the Climbing Stairs Problem II, Missouri Journal of Mathematical Sciences, Vol. 16, No. 1, Winter 2004, pp. 12-17.
   - Mohammad K. Azarian, Fibonacci Identities as Binomial Sums, International Journal of Contemporary Mathematical Sciences, Vol. 7,  No. 38, 2012, pp. 1871-1876.
   - Mohammad K. Azarian, Fibonacci Identities as Binomial Sums II, International Journal of Contemporary Mathematical Sciences, Vol. 7,  No. 42, 2012, pp. 2053-2059.
   - Mohammad K. Azarian, Identities Involving Lucas or Fibonacci and Lucas Numbers as Binomial Sums, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 45, 2012, pp. 2221-2227.
   - P. Bachmann, Niedere Zahlentheorie (1902, 1910), reprinted Chelsea, NY, 1968, vol. 2, p. 70.
   - R. B. Banks, Slicing Pizzas, Racing Turtles and Further Adventures in Applied Mathematics, Princeton Univ. Press, 1999. See p. 84.
   - A. T. Benjamin and J. J. Quinn, Proofs that really count: the art of combinatorial proof, M.A.A. 2003, id. 4.
   - Marjorie Bicknell and Verner E Hoggatt, Fibonacci's Problem Book, Fibonacci Association, San Jose, Calif., 1974.
   - A. Cayley, Theorems in Trigonometry and on Partitions, Messenger of Mathematics, 5 (1876), pp. 164, 188 = Mathematical Papers Vol. 10, n. 634, p. 16.
   - D. Chmiela, K. Kaczmarek, R. Witula, Binomials Transformation Formulas of Scaled Fibonacci Numbers (submitted 2012).
   - B. A. Davey and H. A. Priestley, Introduction to Lattices and Order (2nd edition), CUP, 2002. (See Exercise 1.15.)
   - B. Davis, 'The law of first digits' in 'Science Today'(subsequently renamed '2001') March 1980 p. 55, Times of India, Mumbai.
   - S. R. Finch, Mathematical Constants, Cambridge, 2003, Section 1.2.
   - R. P. Grimaldi, Compositions without the summand 1, Proceedings Thirty-second Southeastern International Conference on Combinatorics, Graph Theory and Computing (Baton Rouge, LA, 2001). Congr. Numer. 152 (2001), 33-43.
   - H. Halberstam and K. F. Roth, Sequences, Oxford, 1966; see Appendix.
   - S. Happersett, "Mathematical meditations", Journal of Mathematics and the Arts, 1 (2007), 29 - 33.
   - G. H. Hardy and E. M. Wright, An Introduction to the Theory of Numbers. 3rd ed., Oxford Univ. Press, 1954; see esp. p. 148.
   - J. Hermes, Anzahl der Zerlegungen einer ganzen rationalen Zahl in Summanden, Math. Ann., 45 (1894), 371-380.
   - V. E. Hoggatt, Jr., Fibonacci and Lucas Numbers. Houghton, Boston, MA, 1969.
   - E. Horowitz and S. Sahni, Fundamentals of Data Structures, Computer Science Press, 1976; p. 338.
   - P. W. Kasteleyn, The statistics of dimers on a lattice. I. The number of dimer arrangements on a quadratic lattice, Physica, 27 (1961), 1209-1225.
   - M. Kauers and P. Paule, The Concrete Tetrahedron, Springer 2011, p. 63.
   - C. Kicey and K. Klimko, Some geometry of Pascal's triangle, Pi Mu Epsilon Journal, 13(4):229-245 (2011).
   - D. E. Knuth, The Art of Computer Programming. Addison-Wesley, Reading, MA, Vol. 1, p. 78; Vol. 3, Section 6.2.1.
   - Thomas Koshy, "Fibonacci and Lucas Numbers with Applications", John Wiley and Sons, 2001.
   - Leonardo of Pisa [Leonardo Pisano], Liber Abaci [The Book of Calculation], 1202.
   - Lukovits et al., Nanotubes: Number of Kekulé structures and aromaticity, J. Chem. Inf. Comput. Sci, (2003), vol. 43, 609-614. See eq. 2 on page 610.
   - I. Lukovits and D. Janezic, "Enumeration of conjugated circuits in nanotubes", J. Chem. Inf. Comput. Sci., vol. 44, 410-414 (2004). See Table 1, second column.
   - B. Malesevic: Some combinatorial aspects of differential operation composition on the space R^n, Univ. Beograd, Publ. Elektrotehn. Fak., Ser. Mat. 9 (1998), 29-33.
   - G. Mantel, Resten van wederkeerige Reeksen, Nieuw Archief v. Wiskunde, 2nd series, I (1894), 172-184.
   - C. N. Menhinick, The Fibonacci Resonance and other new Golden Ratio discoveries, Onperson, (2015), pages 200-206.
   - A. Milicevic and N. Trinajstic, "Combinatorial Enumeration in Chemistry", Chem. Modell., Vol. 4, (2006), pp. 405-469.
   - S. Mneimneh, Fibonacci in The Curriculum: Not Just a Bad Recurrence, in Proceeding SIGCSE '15 Proceedings of the 46th ACM Technical Symposium on Computer Science Education, Pages 253-258.
   - Clifford A. Pickover, A Passion for Mathematics, Wiley, 2005; see p. 49.
   - Clifford A. Pickover, The Math Book, From Pythagoras to the 57th Dimension, 250 Milestones in the History of Mathematics, Sterling Publ., NY, 2009, page 274.
   - A. S. Posamentier & I. Lehmann, The Fabulous Fibonacci Numbers, Prometheus Books, Amherst, NY 2007.
   - P. Ribenboim, The New Book of Prime Number Records, Springer, 1996.
   - J. Riordan, An Introduction to Combinatorial Analysis, Princeton University Press, Princeton, NJ, 1978.
   - A. M. Robert, A Course in p-adic Analysis, Springer-Verlag, 2000; p. 213.
   - J. Roberts, Lure of the Integers, Math. Assoc. America, 1992, p. 288.
   - Manfred R. Schroeder, "Number Theory in Science and Communication", 5th ed., Springer-Verlag, 2009
   - L. E. Sigler, Fibonacci's Liber Abaci, Springer, 2003, pp. 404-405 and [26] on p. 627.
   - Simson, [No first name given], An explanation of an obscure passage in Albert Girard's Commentary ..., Phil. Trans. Royal Soc., 10 (1753), 430-433.
   - N. J. A. Sloane, A Handbook of Integer Sequences, Academic Press, 1973 (includes this sequence).
   - N. J. A. Sloane and Simon Plouffe, The Encyclopedia of Integer Sequences, Academic Press, 1995 (includes this sequence).
   - S. Vajda, Fibonacci and Lucas numbers and the Golden Section, Ellis Horwood Ltd., Chichester, 1989.
   - Van Ravenstein, Tony. "The three gap theorem (Steinhaus conjecture)." Journal of the Australian Mathematical Society (Series A) 45.03 (1988): 360-370.
   - N. N. Vorob'ev, Chisla fibonachchi [Russian], Moscow, 1951. English translation, Fibonacci Numbers, Blaisdell, New York and London, 1961.
   - N. N. Vorobiev, Fibonacci Numbers, Birkhauser (Basel;Boston) 2002.
   - A. Y. Z. Wang, P. Wen, On the partial finite sums of the reciprocals of the Fibonacci numbers, Journal of Inequalities and Applications, 2015; http://www.journalofinequalitiesandapplications.com/content/2015/1/73
   - D. Wells, The Penguin Dictionary of Curious and Interesting Numbers, pp. 61-7, Penguin Books 1987.
   - R. Witula, D. Slota, delta-Fibonacci Numbers, Appl. Anal. Discrete Math., 3 (2009), 310-329.

In [198]:
def search(id=None, seq=None, query=""):

    if id: payload = {"fmt": "json", "q": "id:A{:06d}".format(id)}
    elif seq: payload = {"fmt": "json", "q": ", ".join(map(str,seq))}
    else: payload = {"fmt": "json", "q": query}
    
    doc_result = get("https://oeis.org/search", params=payload,)
    doc = doc_result.json()
    
    results_description = "_Results for query: <a href='{url}'>{url}</a>_<br><hr>".format(url=doc_result.url)
    inner_results = [pretty_print(result) for result in doc['results']]
    
    return Markdown(results_description + "\n<hr>".join(inner_results))

In [197]:
search(seq=[0,1,1,2,3,5,8])

_Results for query: <a href='https://oeis.org/search?q=0%2C+1%2C+1%2C+2%2C+3%2C+5%2C+8&fmt=json'>https://oeis.org/search?q=0%2C+1%2C+1%2C+2%2C+3%2C+5%2C+8&fmt=json</a>_<br><hr><div align='center'>**<a href='http://oeis.org/A000045'>A000045</a>**: _Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1._<br>by _N. J. A. Sloane_, 1964</div>

_Keywords_: `core,nonn,nice,easy,hear,changed`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377 & 610 & 987 & 1597 & 2584 & 4181
\end{array}
$$


_Comments_:
   - Also sometimes called Lamé's sequence.
   - F(n+2) = number of binary sequences of length n that have no consecutive 0's.
   - F(n+2) = number of subsets of {1,2,...,n} that contain no consecutive integers.
   - F(n+1) = number of tilings of a 2 X n rectangle by 2 X 1 dominoes.
   - F(n+1) = number of matchings (i.e., Hosoya index) in a path graph on n vertices: F(5)=5 because the matchings of the path graph on the vertices A, B, C, D are the empty set, {AB}, {BC}, {CD} and {AB, CD}. - _Emeric Deutsch_, Jun 18 2001
   - F(n) = number of compositions of n+1 with no part equal to 1. [Cayley, Grimaldi]
   - Positive terms are the solutions to z = 2*x*y^4 + (x^2)*y^3 - 2*(x^3)*y^2 - y^5 - (x^4)*y + 2*y for x,y >= 0 (Ribenboim, page 193). When x=F(n), y=F(n + 1) and z>0 then z=F(n + 1).
   - For Fibonacci search see Knuth, Vol. 3; Horowitz and Sahni; etc.
   - F(n) is the diagonal sum of the entries in Pascal's triangle at 45 degrees slope. - _Amarnath Murthy_, Dec 29 2001
   - F(n+1) is the number of perfect matchings in ladder graph L_n = P_2 X P_n. - Sharon Sela (sharonsela(AT)hotmail.com), May 19 2002
   - F(n+1) = number of (3412,132)-, (3412,213)- and (3412,321)-avoiding involutions in S_n.
   - This is also the Horadam sequence (0,1,1,1). - _Ross La Haye_, Aug 18 2003
   - An INVERT transform of A019590. INVERT([1,1,2,3,5,8,...]) gives A000129. INVERT([1,2,3,5,8,13,21,...]) gives A028859. - _Antti Karttunen_, Dec 12 2003
   - Number of meaningful differential operations of the k-th order on the space R^3. - _Branko Malesevic_, Mar 02 2004
   - F(n)=number of compositions of n-1 with no part greater than 2. Example: F(4)=3 because we have 3 = 1+1+1 = 1+2 = 2+1.
   - F(n) = number of compositions of n into odd parts; e.g., F(6) counts 1+1+1+1+1+1, 1+1+1+3, 1+1+3+1, 1+3+1+1, 1+5, 3+1+1+1, 3+3, 5+1. - _Clark Kimberling_, Jun 22 2004
   - F(n) = number of binary words of length n beginning with 0 and having all runlengths odd; e.g., F(6) counts 010101, 010111, 010001, 011101, 011111, 000101, 000111, 000001. - _Clark Kimberling_, Jun 22 2004
   - The number of sequences (s(0),s(1),...,s(n)) such that 0<s(i)<5, |s(i)-s(i-1)|=1 and s(0)=1 is F(n+1); e.g., F(5+1) = 8 corresponds to 121212, 121232, 121234, 123212, 123232, 123234, 123432, 123434. - _Clark Kimberling_, Jun 22 2004 [corrected by Neven Juric, Jan 09 2009]
   - Likewise F(6+1) = 13 corresponds to these thirteen sequences with seven numbers: 1212121, 1212123, 1212321, 1212323, 1212343, 1232121, 1232123, 1232321, 1232323, 1232343, 1234321, 1234323, 1234343. - Neven Juric, Jan 09 2008
   - A relationship between F(n) and the Mandelbrot set is discussed in the link "Le nombre d'or dans l'ensemble de Mandelbrot" (in French). - _Gerald McGarvey_, Sep 19 2004
   - For n>0, the continued fraction for F(2n-1)*Phi=[F(2n);L(2n-1),L(2n-1),L(2n-1),...] and the continued fraction for F(2n)*Phi=[F(2n+1)-1;1,L(2n)-2,1,L(2n)-2,...]. Also true: F(2n)*Phi=[F(2n+1);-L(2n),L(2n),-L(2n),L(2n),...] where L(i) is the i-th Lucas number (A000204).... - _Clark Kimberling_, Nov 28 2004 [corrected by _Hieronymus Fischer_, Oct 20 2010]
   - F(n+1) (for n>=1) = number of permutations p of 1,2,3,...,n such that |k-p(k)|<=1 for k=1,2,...,n. (For <=2 and <=3, see A002524 and A002526.) - _Clark Kimberling_, Nov 28 2004
   - The ratios F(n+1)/F(n) for n>0 are the convergents to the simple continued fraction expansion of the golden section. - _Jonathan Sondow_, Dec 19 2004
   - Lengths of successive words (starting with a) under the substitution: {a -> ab, b -> a}. - _Jeroen F.J. Laros_, Jan 22 2005
   - The Fibonacci sequence, like any additive sequence, naturally tends to be geometric with common ratio not a rational power of 10; consequently, for a sufficiently large number of terms, Benford's law of first significant digit (i.e., first digit 1 <= d <= 9 occurring with probability log_10(d+1) - log_10(d)) holds. - _Lekraj Beedassy_, Apr 29 2005
   - a(n) = Sum(abs(A108299(n, k)): 0 <= k <= n). - _Reinhard Zumkeller_, Jun 01 2005
   - a(n) = A001222(A000304(n)).
   - Fib(n+2) = Sum_{k=0..n} binomial(floor((n+k)/2),k), row sums of A046854. - _Paul Barry_, Mar 11 2003
   - Number of order ideals of the "zig-zag" poset. See vol. 1, ch. 3, prob. 23 of Stanley. - _Mitch Harris_, Dec 27 2005
   - F(n+1)/F(n) is also the Farey fraction sequence (see A097545 for explanation) for the golden ratio, which is the only number whose Farey fractions and continued fractions are the same. - _Joshua Zucker_, May 08 2006
   - a(n+2) is the number of paths through 2 plates of glass with n reflections (reflections occurring at plate/plate or plate/air interfaces). Cf. A006356-A006359. - _Mitch Harris_, Jul 06 2006
   - F(n+1) equals the number of downsets (i.e., decreasing subsets) of an n-element fence, i.e., an ordered set of height 1 on {1,2,...,n} with 1 > 2 < 3 > 4 < ... n and no other comparabilities. Alternatively, F(n+1) equals the number of subsets A of {1,2,...,n} with the property that, if an odd k is in A, then the adjacent elements of {1,2,...,n} belong to A, i.e., both k - 1 and k + 1 are in A (provided they are in {1,2,...,n}). - _Brian Davey_, Aug 25 2006
   - Number of Kekulé structures in polyphenanthrenes. See the paper by Lukovits and Janezic for details. - _Parthasarathy Nambi_, Aug 22 2006
   - Inverse: With phi = (sqrt(5) + 1)/2, round(log_phi(sqrt((sqrt(5) a(n) + sqrt(5 a(n)^2 - 4))(sqrt(5) a(n) + sqrt(5 a(n)^2 + 4)))/2)) = n for n >= 3, obtained by rounding the arithmetic mean of the inverses given in A001519 and A001906. - David W. Cantrell (DWCantrell(AT)sigmaxi.net), Feb 19 2007
   - A result of Jacobi from 1848 states that every symmetric matrix over a p.i.d. is congruent to a triple-diagonal matrix. Consider the maximal number T(n) of summands in the determinant of an n X n triple-diagonal matrix. This is the same as the number of summands in such a determinant in which the main-, sub- and super-diagonal elements are all nonzero. By expanding on the first row we see that the sequence of T(n)'s is the Fibonacci sequence without the initial stammer on the 1's. - Larry Gerstein (gerstein(AT)math.ucsb.edu), Mar 30 2007
   - Suppose psi=log(phi). We get the representation F(n)=(2/sqrt(5))*sinh(n*psi) if n is even; F(n)=(2/sqrt(5))*cosh(n*psi) if n is odd. There is a similar representation for Lucas numbers (A000032). Many Fibonacci formulas now easily follow from appropriate sinh and cosh formulas. For example: the de Moivre theorem (cosh(x)+sinh(x))^m=cosh(mx)+sinh(mx) produces L(n)^2+5F(n)^2=2L(2n) and L(n)F(n)=F(2n) (setting x=n*psi and m=2). - _Hieronymus Fischer_, Apr 18 2007
   - Inverse: floor(log_phi(sqrt(5)*Fib(n))+1/2)=n, for n>1. Also for n>0, floor(1/2*log_phi(5*Fib(n)*Fib(n+1)))=n. Extension valid for integer n, except n=0,-1: floor(1/2*sign(Fib(n)*Fib(n+1))*log_phi|5*Fib(n)*Fib(n+1)|)=n (where sign(x) = sign of x). - _Hieronymus Fischer_, May 02 2007
   - F(n+2) = The number of Khalimsky-continuous functions with a two-point codomain. - Shiva Samieinia (shiva(AT)math.su.se), Oct 04 2007
   - From Kauffman and Lopes, Proposition 8.2, p. 21: "The sequence of the determinants of the Fibonacci sequence of rational knots is the Fibonacci sequence (of numbers)." - _Jonathan Vos Post_, Oct 26 2007
   - This is a_1(n) in the Doroslovacki reference.
   - Let phi = (sqrt(5)+1)/2 = 1.6180339...; then phi^n = (1/phi)*a(n) + a(n+1). Example: phi^4 = 6.8541019... = (0.6180339...)*3 + 5. Also phi = 1/1 + 1/2 + 1/(2*5) + 1/(5*13) + 1/(13*34) + 1/(34*89) + ... - _Gary W. Adamson_, Dec 15 2007
   - The sequence of first differences, Fib(n+1)-Fib(n), is essentially the same sequence: 1, 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ... - _Colm Mulcahy_, Mar 03 2008
   - a(n)= the number of different ways to run up a staircase with n steps, taking steps of odd sizes where the order is relevant and there is no other restriction on the number or the size of each step taken. - _Mohammad K. Azarian_, May 21 2008
   - Equals row sums of triangle A144152. - _Gary W. Adamson_, Sep 12 2008
   - Except for the initial term, the numerator of the convergents to the recursion x = 1/(x+1). - _Cino Hilliard_, Sep 15 2008
   - F(n) is the number of possible binary sequences of length n that obey the sequential construction rule: if last symbol is 0, add the complement (1); else add 0 or 1. Here 0,1 are metasymbols for any 2-valued symbol set. This rule has obvious similarities to JFJ Laros's rule, but is based on addition rather than substitution and creates a tree rather than a single sequence. - _Ross Drewe_, Oct 05 2008
   - F(n) = Product_{k=1..(n-1)/2} (1 + 4*cos^2 k*Pi/n), where terms = roots to the Fibonacci product polynomials, A152063. - _Gary W. Adamson_, Nov 22 2008
   - Fp == 5^((p-1)/2) mod p, p = prime [Schroeder, p. 90]. - _Gary W. Adamson_ & _Alexander R. Povolotsky_, Feb 21 2009
   - (Ln)^2 - 5*(Fn)^2 = 4*(-1)^n. Example: 11^2 - 5*5 = -4. - _Gary W. Adamson_, Mar 11 2009
   - Output of Kasteleyn's formula for the number of perfect matchings of an m X n grid specializes to the Fibonacci sequence for m=2. - Sarah-Marie Belcastro (smbelcas(AT)toroidalsnark.net), Jul 04 2009
   - (Fib(n),Fib(n+4)) satisfies the Diophantine equation: X^2 + Y^2 - 7XY = 9*(-1)^n. - Mohamed Bouhamida (bhmd95(AT)yahoo.fr), Sep 06 2009
   - (Fib(n),Fib(n+2)) satisfies the Diophantine equation: X^2 + Y^2 - 3XY = (-1)^n. - Mohamed Bouhamida (bhmd95(AT)yahoo.fr), Sep 08 2009
   - a(n+2)=A083662(A131577(n)). - _Reinhard Zumkeller_, Sep 26 2009
   - Difference between of number of closed walks of length n+1 from a node on a pentagon and number of walks of length n+1 between two adjacent nodes on a pentagon. - _Henry Bottomley_, Feb 10 2010
   - F(n+1) = number of Motzkin paths of length n having exactly one weak ascent. A Motzkin path of length n is a lattice path from (0,0) to (n,0) consisting of U=(1,1), D=(1,-1) and H=(1,0) steps and never going below the x-axis. A weak ascent in a Motzkin path is a maximal sequence of consecutive U and H steps. Example: a(5)=5 because we have (HHHH), (HHU)D, (HUH)D, (UHH)D, and (UU)DD (the unique weak ascent is shown between parentheses; see A114690). - _Emeric Deutsch_, Mar 11 2010
   - (F(n-1) + F(n+1))^2 - 5F(n-2)*F(n+2) = 9*(-1)^n. - Mohamed Bouhamida (bhmd95(AT)yahoo.fr), Mar 31 2010
   - From the Pinter and Ziegler reference's abstract: authors "show that essentially the Fibonacci sequence is the unique binary recurrence which contains infinitely many three-term arithmetic progressions. A criterion for general linear recurrences having infinitely many three-term arithmetic progressions is also given." - _Jonathan Vos Post_, May 22 2010
   - F(n+1) = number of paths of length n starting at initial node on the path graph P_4. - _Johannes W. Meijer_, May 27 2010
   - F(k) = Number of cyclotomic polynomials in denominator of generating function for number of ways to place k nonattacking queens on an n X n board. - _Vaclav Kotesovec_, Jun 07 2010
   - As n-> inf., (a(n)/a(n-1) - a(n-1)/a(n)) tends to 1.0. Example: a(12)/a(11) - a(11)/a(12) = 144/89 - 89/144 = 0.99992197.... - _Gary W. Adamson_, Jul 16 2010
   - From _Hieronymus Fischer_, Oct 20 2010: (Start)
   - Fibonacci numbers are those numbers m such that m*phi is closer to an integer than k*phi for all k, 1<=k<m. More formally: a(0)=0, a(1)=1, a(2)=1, a(n+1)=minimal m>a(n) such that m*phi is closer to an integer than a(n)*phi.
   - For all numbers 1<=k<Fib(n), the inequality |k*phi-round(k*phi)| > |Fib(n)*phi-round(Fib(n)*phi)| holds.
   - Fib(n)*phi - round(Fib(n)*phi) = -((-phi)^(-n)), for n>1.
   - Fract(0.5+Fib(n)*phi) = 0.5 -(-phi)^(-n), for n>1.
   - Fract(Fib(n)*phi) = (1/2)*(1+(-1)^n)-(-phi)^(-n), n>1.
   - Inverse: n = -log_phi |0.5-fract(0.5+Fib(n)*phi)|.
   - (End)
   - F(A001177(n)*k) mod n = 0, for any integer k. - _Gary Detlefs_, Nov 27 2010
   - F(n+k)^2-F(n)^2 = F(k)*F(2n+k), for even k. - _Gary Detlefs_, Dec 04 2010
   - F(n+k)^2+F(n)^2 = F(k)*F(2n+k), for odd k. - _Gary Detlefs_, Dec 04 2010
   - F(n) = round(phi* F(n-1)) for n>1. - _Joseph P. Shoulak_, Jan 13 2012
   - For n > 0: a(n) = length of n-th row in Wythoff array A003603. - _Reinhard Zumkeller_, Jan 26 2012
   - From _Bridget Tenner_, Feb 22 2012: (Start)
   - The number of free permutations of [n].
   - The number of permutations of [n] for which s_k in supp(w) implies s_{k+-1} not in supp(w).
   - The number of permutations of [n] in which every decomposition into length(w) reflections is actually composed of simple reflections. (End)
   - The sequence F(n+1)^(1/n) is increasing. The sequence F(n+2)^(1/n) is decreasing. - _Thomas Ordowski_, Apr 19 2012
   - Two conjectures: For n > 1, F(n+2)^2 mod F(n+1)^2 = F(n)*F(n+1) - (-1)^n. For n > 0, (F(2n) + F(2n+2))^2 = F(4n+3) + sum_{k = 2..2n}F(2k). - _Alex Ratushnyak_, May 06 2012
   - From _Ravi Kumar Davala_, Jan 30 2014: (Start)
   - Proof of Ratushnyak's first conjecture: For n > 1, F(n+2)^2 - F(n)*F(n+1) + (-1)^n = 2F(n+1)^2.
   - Consider: F(n+2)^2 - F(n)*F(n+1) - 2F(n+1)^2
   -          = F(n+2)^2 - F(n+1)^2 - F(n+1)^2 - F(n)*F(n+1)
   -          =(F(n+2) + F(n+1))*(F(n+2) - F(n+1)) - F(n+1)*(F(n+1) + F(n))
   -          = F(n+3)*F(n) - F(n+1)*F(n+2) = -(-1)^n.
   - Proof of second conjecture: L(n) stands for Lucas number sequence from A000032.
   - Consider the fact that
   -     L(2n+1)^2 = L(4n+2) - 2
   -    (F(2n) + F(2n+2))^2 = F(4n+1) + F(4n+3) - 2
   -    (F(2n) + F(2n+2))^2 = sum{k = 2..2n, F(2k)} + F(4n+3).
   - (End)
   - The relationship: INVERT transform of (1,1,0,0,0,...) = (1, 2, 3, 5, 8,...), while the INVERT transform of (1,0,1,0,1,0,1,...) = (1, 1, 2, 3, 5, 8,...) is equivalent to: The numbers of compositions using parts 1 and 2 is equivalent to the numbers of compositions using parts == 1 mod 2 (i.e., the odd integers). Generally, the numbers of compositions using parts 1 and k is equivalent to the numbers of compositions of (n+1) using parts 1 mod k. Cf. A000930 for k = 3 and A003269 for k = 4. Example: for k = 2, n = 4 we have the compositions (22; 211, 121; 112; 1111) = 5; but using parts 1 and 3 we have for n = 5: (311, 131, 113, 11111, 5) = 5. - _Gary W. Adamson_, Jul 05 2012
   - The sequence F(n) is the binomial transformation of the alternating sequence (-1)^(n-1)*F(n), whereas the sequence F(n+1) is the binomial transformation of the alternating sequence (-1)^n*F(n-1). Both of these facts follow easily from the equalities a(n;1)=F(n+1) and b(n;1)=F(n) where a(n;d) and b(n;d) are so-called "delta-Fibonacci" numbers as defined in comments to A014445 (see also the papers of Witula et al.). - _Roman Witula_, Jul 24 2012
   - F(n) is the number of different (n-1)-digit binary numbers such that all substrings of length > 1 have at least one digit equal to 1. Example: for n = 5 there are 8 binary numbers with n - 1 = 4 digits (1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111), only the F(n) = 5 numbers 1010, 1011, 1101, 1110 and 1111 have the desired property. - _Hieronymus Fischer_, Nov 30 2012
   - For positive n, F(n+1) equals the determinant of the n X n tridiagonal matrix with 1's along the main diagonal, i's along the superdiagonal and along the subdiagonal where i = sqrt(-1). Example: Det([1,i,0,0; i,1,i,0; 0,i,1,i; 0,0,i,1]) = F(4+1) = 5. - _Philippe Deléham_, Feb 24 2013
   - For n>=1, number of compositions of n where there is a drop between every second pair of parts, starting with the first and second part; see example. Also, a(n+1) is the number of compositions where there is a drop between every second pair of parts, starting with the second and third part; see example. - _Joerg Arndt_, May 21 2013
   - Central terms of triangles in A162741 and A208245, n > 0. - _Reinhard Zumkeller_, Jul 28 2013
   - For n>=4, F(n-1) is the number of simple permutations in the geometric grid class given in A226433. - _Jay Pantone_, Sep 08 2013
   - a(n) are the pentagon (not pentagonal) numbers because the algebraic degree 2 number rho(5) = 2*cos(Pi/5) = phi (golden section), the length ratio diagonal/side in a pentagon, has minimal polynomial C(5,x) = x^2 - x - 1 (see A187360, n=5), hence rho(5)^n = a(n-1)*1 + a(n)*rho(5), n >= 0, in the power basis of the algebraic number field Q(rho(5)). One needs a(-1) = 1 here. See also the P. Steinbach reference under A049310. - _Wolfdieter Lang_, Oct 01 2013
   - A010056(a(n)) = 1. - _Reinhard Zumkeller_, Oct 10 2013
   - Define F(-n) to be F(n) for n odd and -F(n) for n even. Then for all n and k, F(n+2k)^2 - F(n)^2 = F(n+k)*( F(n+3k) - F(n-k) ). - _Charlie Marion_, Dec 20 2013
   - ( F(n), F(n+2k) ) satisfies the Diophantine equation: X^2 + Y^2 - L(2k)*X*Y = F(4k)^2*(-1)^n. This generalizes Bouhamida’s comments dated Sep 06 2009 and Sep 08 2009. - _Charlie Marion_, Jan 07 2014
   - For any prime p there is an infinite periodic subsequence within F(n) divisible by p, that begins at index n = 0 with value 0, and its first nonzero term at n = A001602(i), and period k = A001602(i). Also see A236479. - _Richard R. Forberg_, Jan 26 2014
   - Range of row n of the circular Pascal array of order 5. - _Shaun V. Ault_, May 30 2014 [orig. Kicey-Klimko 2011, and observations by Glen Whitehead; more general work found in Ault-Kicey 2014]
   - Nonnegative range of the quintic polynomial 2*y - y^5 + 2*x*y^4 + x^2*y^3 - 2*x^3*y^2 - x^4*y with x, y >= 0, see Jones 1975. - _Charles R Greathouse IV_, Jun 01 2014
   - The expression round(1/(F(k+1)/F(n) + F(k)/F(n+1))), for n > 0, yields a Fibonacci sequence with k-1 leading zeros (with rounding 0.5 to 0). - _Richard R. Forberg_, Aug 04 2014
   - Conjecture: For n > 0, F(n) is the number of all admissible residue classes for which specific finite subsequences of the Collatz 3n + 1 function consists of n+2 terms. This has been verified for 0 < n < 51. For details see Links. - _Mike Winkler_, Oct 03 2014
   - a(4)=3 and a(6)=8 are the only Fibonacci numbers that are of the form prime+1. - _Emmanuel Vantieghem_, Oct 02 2014
   - a(1)=1=a(2), a(3)=2 are the only Fibonacci numbers that are of the form prime-1. - _Emmanuel Vantieghem_, Jun 07 2015
   - Any consecutive pair (m, k) of the Fibonacci sequence a(n) illustrates a fair equivalence between m miles and k kilometers. For instance, 8 miles ~ 13 km; 13 miles ~ 21 km. -_Lekraj Beedassy_, Oct 06 2014
   - (n -> oo) lim (log F(n+1)/log F(n))^n = e. - _Thomas Ordowski_, Oct 06 2014
   - a(n+1) counts closed walks on K_2, containing one loop on the other vertex. Equivalently the (1,1)_entry of A^(n+1) where the adjacency matrix of digraph is A=(0,1; 1,1). - _David Neil McGrath_, Oct 29 2014
   - a(n-1) counts closed walks on the graph G(1-vertex;l-loop,2-loop). - _David Neil McGrath_, Nov 26 2014
   - From _Tom Copeland_, Nov 02 2014: (Start)
   - Let P(x) = x/(1+x) with comp. inverse Pinv(x) = x/(1-x) = -P[-x], and C(x)= [1-sqrt(1-4x)]/2, an o.g.f. for the shifted Catalan numbers A000108, with inverse Cinv(x) = x * (1-x).
   - Fin(x) = P[C(x)] = C(x)/[1 + C(x)] is an o.g.f. for the Fine numbers, A000957 with inverse Fin^(-1)(x) = Cinv[Pinv(x)] = Cinv[-P(-x)].
   - Mot(x) = C[P(x)] = C[-Pinv(-x)] gives an o.g.f. for shifted A005043, the Motzkin or Riordan numbers with comp. inverse Mot^(-1)(x) = Pinv[Cinv(x)] = (x - x^2) / (1 - x + x^2) (cf. A057078).
   - BTC(x) = C[Pinv(x)] gives A007317, a binomial transform of the Catalan numbers, with BTC^(-1)(x) = P[Cinv(x)].
   - Fib(x) = -Fin[Cinv(Cinv(-x))] = -P[Cinv(-x)] = x + 2 x^2 + 3 x^3 + 5 x^4 + ... = (x+x^2)/[1-x-x^2] is an o.g.f. for the shifted Fibonacci sequence A000045, so the comp. inverse is Fib^(-1)(x) = -C[Pinv(-x)] = -BTC(-x) and Fib(x) = -BTC^(-1)(-x).
   - Generalizing to P(x,t) = x /(1 + t*x) and Pinv(x,t) = x /(1 - t*x) = -P(-x,t) gives other relations to lattice paths, such as the o.g.f. for A091867, C[P[x,1-t]], and that for A104597, Pinv[Cinv(x),t+1].
   - (End)
   - In keeping with historical accounts (see the references by P. Singh and S. Kak), the generalized Fibonacci sequence a, b, a + b, a + 2b, 2a + 3b, 3a + 5b, ... can also be described as the Gopala-Hemachandra numbers H(n) = H(n-1) + H(n-2), with F(n) = H(n) for a = b = 1, and Lucas sequence L(n) = H(n) for a = 2, b = 1. - _Lekraj Beedassy_, Jan 11 2015
   - D. E. Knuth writes: "Before Fibonacci wrote his work, the sequence F_{n} had already been discussed by Indian scholars, who had long been interested in rhythmic patterns that are formed from one-beat and two-beat notes. The number of such rhythms having n beats altogether is F_{n+1}; therefore both Gopāla (before 1135) and Hemachandra (c. 1150) mentioned the numbers 1, 2, 3, 5, 8, 13, 21, ... explicitly." (TAOCP Vol. 1, 2nd ed.) - _Peter Luschny_, Jan 11 2015
   - F(n+1) equals the number of binary words of length n avoiding runs of zeroes of odd lengths. - _Milan Janjic_, Jan 28 2015
   - From _Russell Jay Hendel_, Apr 12 2015: (Start)
   - We prove Conjecture 1 of Rashid listed in the Formula section.
   - We use the following notation: F(n)=A000045(n), the Fibonacci numbers, and L(n) = A000032(n), the Lucas numbers. The fundamental Fibonacci-Lucas recursion asserts that G(n) = G(n-1)+ G(n-2), with "L" or "F" replacing "G".
   - We need the following prerequisites which we label (A), (B),(C), (D). The prerequisites are formulas in the Koshy book listed in the References section. (A) F(m-1)+F(m+1) = L(m) (Koshy, p. 97, #32), (B) L(2m)+2(-1)^m = L(m)^2 (Koshy p. 97, #41), (C) F(m+k)F(m-k) = (-1)^n F(k)^2 (Koshy, p. 113, #24, Tagiuri's identity), and (D) F(n)^2+F(n+1)^2 = F(2n+1) (Koshy, p. 97, #30).
   - We must also prove (E), L(n+2) F(n-1) = F(2n+1)+2(-1)^n. To prove (E), first note that by (A), proof of (E) is equivalent to proving that F(n+1)F(n-1) + F(n+3)F(n-1) = F(2n+1)+2(-1)^n. But by (C) with k=1, we have F(n+1)F(n-1) = F(n)^2 +(-1)^n. Applying (C) again with k=2 and m=n+1, we have F(n+3)F(n-1) = F(n+1)+(-1)^n. Adding these two applications of (C) together and using (D) we have, F(n+1)F(n-1) + F(n+3)F(n-1) = F(n)^2 + F(n+1)^2 + 2(-1)^n = F(2n+1)+2(-1)^n, completing the proof of (E).
   - We now prove Conjecture 1. By (A) and the Fibonacci-Lucas recursion, we have F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4) = [F(2n+1)+F(2n+3)] + [F(2n+2)+F(2n+4)] = L(2n+2)+L(2n+3)=L(2n+4). But then by (B), with m=2n+4, we have sqrt(L(2n+4)+2(-1)^n)) = L(n+2). Finally by (E), we have L(n+2) F(n-1)= F(2n+1)+2*(-1)^n. Dividing both sides by F(n-1), we have (F(2n+1)+2*(-1)^n)/F(n-1) = L(n+2) = sqrt(F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4)+2(-1)^n), as required.
   - (End)
   - In Fibonacci's Liber Abaci the rabbit problem appears in the translation of L. E. Sigler on pp. 404-405, and a remark [27] on p. 637. - _Wolfdieter Lang_, Apr 17 2015
   - a(n) counts partially ordered partitions of (n-1) into parts 1,2,3 where only the order of adjacent 1's and 2's are unimportant. (See example.) - _David Neil McGrath_, Jul 27 2015
   - F(n) divides F(nk). Proved by Marjorie Bicknell and Verner E Hoggatt Jr. - _Juhani Heino_, Aug 24 2015
   - F(n) is the number of UDU-equivalence classes of ballot paths of length n. Two ballot paths of length n with steps U = (1,1), D = (1,-1) are UDU-equivalent whenever the positions of UDU are the same in both paths. - _Kostas Manes_, Aug 25 2015
   - Cassini's identity F(2n+1) * F(2n+3) = F(2n+2)^2 + 1 is the basis for a geometrical paradox (or dissection fallacy) in A262342. - _Jonathan Sondow_, Oct 23 2015
   - For n >= 4, F(n) is the number of up-down words on alphabet {1,2,3} of length n-2. - _Ran Pan_, Nov 23 2015
   - F(n+2) is the number of terms in p(n), where p(n)/q(n) is the n-th convergent of the formal infinite continued fraction [a(0),a(1),...]; e.g., p(3) = a(0)a(1)a(2)a(3) + a(0)a(1) + a(0)a(3) + a(2)a(3) + 1 has F(5) terms. Also, F(n+1) is the number of terms in q(n). - _Clark Kimberling_, Dec 23 2015
   - F(n+1) (for n>=1) is the permanent of an n X n matrix M with M(i,j)=1 if |i-j|<=1 and 0 otherwise. - _Dmitry Efimov_, Jan 08 2016
   - A trapezoid has three sides of lengths in order F(n), F(n+2), F(n). For increasing n a very close approximation to the maximum area will have the fourth side equal to 2*F(n+1). For a trapezoid with lengths of sides in order F(n+2), F(n), F(n+2), the fourth side will be F(n+3). - _J. M. Bergot_, Mar 17 2016
   - (1) Join two triangles with lengths of sides L(n), F(n+3), L(n+2) and F(n+2), L(n+1), L(n+2) (where L(n)=A000032(n)) along the common side of length L(n+2) to create an irregular quadrilateral. Its area is approximately (5*F(2*n-1) - (F(2*n-7) - F(2*n-13))/5. (2) Join two triangles with lengths of sides L(n), F(n+2), F(n+3) and L(n+1), F(n+1, F(n+3) along the common side F(n+3) to form an irregular quadrilateral. Its area is approximately 4*F(2*n-1) - 2*(F(2*n-7) + F(2*n-18)). - _J. M. Bergot_, Apr 06 2016
   - From _Clark Kimberling_, Jun 13 2016: (Start)
   - Let T* be the infinite tree with root 0 generated by these rules: if p is in T*, then p+1 is in T* and x*p is in T*.
   - Let g(n) be the set of nodes in the n-th generation, so that g(0) = {0}, g(1) = {1}, g(2) = {2, x}, g(3) = {3, 2x, x+1, x^2}, etc.
   - Let T(r) be the tree obtained by substituting r for x.
   - If a positive integer N is not a square and r = sqrt(N), then the number of (not necessarily distinct) integers in g(n) is A000045(n), for n > = 1. See A274142. (End)
   - Consider the partitions of n, with all summands initially listed in nonincreasing order. Freeze all the 1's in place and then allow all the other summands to change their order, without displacing any of the 1's. The resulting number of arrangements is a(n+1). - _Gregory L. Simay_, Jun 14 2016

_Formulae_:
   - G.f.: x / (1 - x - x^2).
   - G.f.: Sum_{n>=0} x^n * Product_{k=1..n} (k + x)/(1 + k*x). - _Paul D. Hanna_, Oct 26 2013
   - F(n) = ((1+sqrt(5))^n-(1-sqrt(5))^n)/(2^n*sqrt(5)).
   - Alternatively, F(n) = ((1/2+sqrt(5)/2)^n-(1/2-sqrt(5)/2)^n)/sqrt(5).
   - F(n) = F(n-1) + F(n-2) = -(-1)^n F(-n).
   - F(n) = round(phi^n/sqrt(5)).
   - F(n+1) = Sum_{j=0..[n/2]} binomial(n-j, j).
   - This is a divisibility sequence; that is, if n divides m, then a(n) divides a(m). - _Michael Somos_, Apr 07 2012
   - E.g.f.: (2/sqrt(5))*exp(x/2)*sinh(sqrt(5)*x/2). - _Len Smiley_, Nov 30 2001
   - [0 1; 1 1]^n [0 1] = [F(n); F(n+1)]
   - x | F(n) ==> x | F(kn).
   - A sufficient condition for F(m) to be divisible by a prime p is (p - 1) divides m, if p == 1 or 4 (mod 5); (p + 1) divides m, if p == 2 or 3 (mod 5); or 5 divides m, if p = 5. (This is essentially Theorem 180 in Hardy and Wright.) - Fred W. Helenius (fredh(AT)ix.netcom.com), Jun 29 2001
   - a(n)=F(n) has the property: F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1). - _Miklos Kristof_, Nov 13 2003
   - _Kurmang. Aziz. Rashid_, Feb 21 2004, makes 4 conjectures and gives 3 theorems:
   - Conjecture 1: for n>=2 sqrt{F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4)+2*(-1)^n}={F(2n+1)+2*(-1)^n}/F(n-1). [For a proof see Comments section.]
   - Conjecture 2: for n>=0, {F(n+2)* F(n+3)}-{F(n+1)* F(n+4)}+ (-1)^n = 0.
   - Conjecture 3: for n>=0, F(2n+1)^3 - F(2n+1)*[(2*A^2)-1] - [A + A^3]=0, where A = {F(2n+1)+sqrt{5*F(2n+1)^2 +4}}/2.
   - Conjecture 4: for x>=5, if x is a Fibonacci number >= 5 then g*x*[{x+sqrt{5*(x^2) +- 4}}/2]*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[2x+{{3x+3*sqrt {5*(x^2) +- 4}}/2}]^2+[2x+{{x+sqrt{5*(x^2) +- 4}}/2}] +- x*[2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2 -x*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[x+{{x+sqrt{5*(x^2) +- 4}}/2}]* [2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2= 0, where g = {1 + sqrt(5)/2}.
   - Theorem 1: for n>=0, {F(n+3)^ 2 - F(n+1)^ 2}/F(n+2)={F(n+3)+ F(n+1)}.
   - Theorem 2: for n>=0, F(n+10) = 11*F(n+5) + F(n).
   - Theorem 3: for n>=6, F(n) = 4*F(n-3) + F(n-6).
   - Conjecture 2 of Rashid is actually a special case of the general law F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1) (take n <- n+1 and m <- -(n+4) in this law). - Harmel Nestra (harmel.nestra(AT)ut.ee), Apr 22 2005
   - Conjecture 2 of Rashid Kurmang simplified: F(n)*F(n+3) = F(n+1)*F(n+2)-(-1)^n. Follows from d'Ocagne's identity: m=n+2. - _Alex Ratushnyak_, May 06 2012
   - Conjecture: for all c such that 2-Phi <= c < 2*(2-Phi) we have F(n) = floor(Phi*a(n-1)+c) for n > 2. - _Gerald McGarvey_, Jul 21 2004
   - |2*Fib(n) - 9*Fib(n+1)| = 4*A000032(n) + A000032(n+1). - _Creighton Dement_, Aug 13 2004
   - For x > Phi, Sum_{n=0..inf} F(n)/x^n = x/(x^2 - x - 1) - _Gerald McGarvey_, Oct 27 2004
   - F(n+1) = exponent of the n-th term in the series f(x, 1) determined by the equation f(x, y) = xy + f(xy, x). - _Jonathan Sondow_, Dec 19 2004
   - a(n-1) = Sum_{k=0..n} (-1)^k*binomial(n-ceil(k/2), floor(k/2)). - _Benoit Cloitre_, May 05 2005
   - F(n+1) = Sum_{k=0..n} binomial((n+k)/2, (n-k)/2)(1+(-1)^(n-k))/2. - _Paul Barry_, Aug 28 2005
   - Fibonacci(n) = Product(1 + 4[cos(j*Pi/n)]^2, j=1..ceil(n/2)-1). [Bicknell and Hoggatt, pp. 47-48.] - _Emeric Deutsch_, Oct 15 2006
   - F(n) = 2^-(n-1)*Sum_{k=0..floor((n-1)/2)} binomial(n,2*k+1)*5^k. - _Hieronymus Fischer_, Feb 07 2006
   - a(n) = (b(n+1)+b(n-1))/n where {b(n)} is the sequence A001629. - _Sergio Falcon_, Nov 22 2006
   - F(n*m) = Sum_{k = 0..m} binomial(m,k)*F(n-1)^k*F(n)^(m-k)*F(m-k). The generating function of F(n*m) (n fixed, m = 0,1,2...) is G(x) = F(n)*x / ((1-F (n-1)*x)^2-F(n)*x*(1-F(n-1)*x)-( F(n)*x)^2). E.g., F(15) = 610 = F(5*3) = binomial(3,0)* F(4)^0*F(5)^3*F(3) + binomial(3,1)* F(4)^1*F(5)^2*F(2) + binomial(3,2)* F(4)^2*F(5)^1*F(1) + binomial(3,3)* F(4)^3*F(5)^0*F(0) = 1*1*125*2 + 3*3*25*1 + 3*9*5*1 + 1*27*1*0 = 250 + 225 + 135 + 0 = 610. - _Miklos Kristof_, Feb 12 2007
   - From _Miklos Kristof_, Mar 19 2007: (Start)
   -   Let L(n) = A000032(n) = Lucas numbers. Then:
   -   For a>=b and odd b, F(a+b)+F(a-b)=L(a)*F(b).
   -   For a>=b and even b, F(a+b)+F(a-b)=F(a)*L(b).
   -   For a>=b and odd b, F(a+b)-F(a-b)=F(a)*L(b).
   -   For a>=b and even b, F(a+b)-F(a-b)=L(a)*F(b).
   -   F(n+m)+(-1)^m*F(n-m)=F(n)*L(m);
   -   F(n+m)-(-1)^m*F(n-m)=L(n)*F(m);
   -   F(n+m+k)+(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =F(n)*L(m)*L(k);
   -   F(n+m+k)-(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =L(n)*L(m)*F(k);
   -   F(n+m+k)+(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =L(n)*F(m)*L(k);
   -   F(n+m+k)-(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =5*F(n)*F(m)*F(k). (End)
   - A corollary to Kristof 2007 is 2*F(a+b)=F(a)*L(b)+L(a)*F(b). - _Graeme McRae_, Apr 24 2014
   - For n>m, the sum of the 2m consecutive Fibonacci numbers F(n-m-1) thru F(n+m-2) is F(n)*L(m) if m is odd, and L(n)*F(m) if m is even (see the McRae link). - _Graeme McRae_, Apr 24 2014.
   - Fib(n) = b(n)+(p-1)*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital sum analog of the Fibonacci recurrence, defined by b(k)=ds_p(b(k-1))+ds_p(b(k-2)), b(0)=0, b(1)=1, ds_p=digital sum base p. Example for base p=10: Fib(n)=A010077(n)+9*Sum_{1<k<n} A059995(A010077(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - Fib(n) = b(n)+p*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital product analog of the Fibonacci recurrence, defined by b(k)=dp_p(b(k-1))+dp_p(b(k-2)), b(0)=0, b(1)=1, dp_p=digital product base p. Example for base p=10: Fib(n)=A074867(n)+10*Sum_{1<k<n} A059995(A074867(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - a(n) = denominator of continued fraction [1,1,1,...] (with n ones); e.g., 2/3 = continued fraction [1,1,1]; where barover[1] = [1,1,1...] = 0.6180339.... - _Gary W. Adamson_, Nov 29 2007
   - F(n + 3) = 2F(n + 2) - F(n), F(n + 4) = 3F(n + 2) - F(n), F(n + 8) = 7F(n + 4) - F(n), F(n + 12) = 18F(n + 6) - F(n). - _Paul Curtz_, Feb 01 2008
   - 1 = 1/(1*2) + 1/(1*3) + 1/(2*5) + 1/(3*8) + 1/(5*13) + ... = 1/2 + 1/3 + 1/10 + 1/24 + 1/65 + 1/168 + ...; where A059929 = (0, 2, 3, 10, 24, 65, 168,...). - _Gary W. Adamson_, Mar 16 2008
   - a(2^n) = prod{i=0}^{n-2}B(i) where B(i) is A001566. Example 3*7*47 = Fib(16). - _Kenneth J Ramsey_, Apr 23 2008
   - F(n) = (1/(n-1)!) * (n^(n-1) - (C(n-2,0) + 4*C(n-2,1) + 3*C(n-2,2))*n^(n-2) + (10*C(n-3,0) + 49*C(n-3,1) + 95*C(n-3,2) + 83*C(n-3,3) + 27*C(n-3,4))*n^(n-3) - (90*C(n-4,0) + 740*C(n-4,1) + 2415*C(n-4,2) + 4110*C(n-4,3) + 3890*C(n-4,4) + 1950*C(n-4,5) + 405*C(n-4,6))*n^(n-4) + ... ). - _André F. Labossière_, Nov 24 2004
   - a(n+1) = Sum_{k, 0<=k<=n} A109466(n,k)*(-1)^(n-k). -_Philippe Deléham_, Oct 26 2008
   - a(n) = Sum_{l_1=0..n+1} Sum_{l_2=0..n}...Sum_{l_i=0..n-i}... Sum_{l_n=0..1} delta(l_1,l_2,...,l_i,...,l_n), where delta(l_1,l_2,...,l_i,...,l_n) = 0 if any l_i + l_(i+1) >= 2 for i=1..n-1 and delta(l_1,l_2,...,l_i,...,l_n) = 1 otherwise. - _Thomas Wieder_, Feb 25 2009
   - a(n+1) = 2^n sqrt(Product_{k=1..n} cos(k Pi/(n+1))^2+1/4)) (Kasteleyn's formula specialized). - Sarah-Marie Belcastro (smbelcas(AT)toroidalsnark.net), Jul 04 2009
   - a(n+1) = Sum_{k=floor[n/2] mod 5} C(n,k) - Sum_{k=floor[(n+5)/2] mod 5} C(n,k) = A173125(n) - A173126(n) = |A054877(n)-A052964(n-1)|. - _Henry Bottomley_, Feb 10 2010
   - If p[i]=modp(i,2) and if A is Hessenberg matrix of order n defined by: A[i,j]=p[j-i+1], (i<=j), A[i,j]=-1, (i=j+1), and A[i,j]=0 otherwise. Then, for n>=1, a(n)=det A. - _Milan Janjic_, May 02 2010
   - Limit(F(k+n)/F(k), k = infinity) = (L(n) + F(n)*sqrt(5))/2 with the Lucas numbers L(n)= A000032(n). - _Johannes W. Meijer_, May 27 2010
   - For n>=1, F(n)=round(log_2(2^{phi*F(n-1)} + 2^{phi*F(n-2)})), where phi is the golden ratio. - _Vladimir Shevelev_, Jun 24 2010, Jun 27 2010
   - For n>=1, a(n+1)=ceil(phi*a(n)), if n is even and a(n+1)=floor(phi*a(n)), if n is odd (phi = golden ratio). - _Vladimir Shevelev_, Jul 01 2010
   - a(n) = 2*a(n-2) + a(n-3), n>2. - _Gary Detlefs_, Sep 08 2010
   - a(2^n) = Prod_{i=0..n-1} A000032(2^i). - _Vladimir Shevelev_, Nov 28 2010
   - a(n)^2 - a(n-1)^2 = a(n+1)*a(n-2), see A121646.
   - a(n) = sqrt((-1)^k*(a(n+k)^2 - a(k)*a(2n+k))), for any k. - _Gary Detlefs_, Dec 03 2010
   - F(2*n) = F(n+2)^2 - F(n+1)^2 - 2*F(n)^2. - _Richard R. Forberg_, Jun 04 2011
   - (-1)^(n+1) = F(n)^2 + F(n)*F(1+n) - F(1+n)^2.
   -   F(n) = -F(n+2)(-2 + (F(n+1))^4 + 2*(F(n+1)^3*F(n+2)) - (F(n+1)*F(n+2))^2 2*F(n+1)(F(n+2))^3 + (F(n+2))^4)- F(n+1). - _Artur Jasinski_, Nov 17 2011
   - F(n) = 1 + Sum_{x=1..n-2} F(x). - _Joseph P. Shoulak_, Feb 05 2012
   - F(n) = 4*F(n-2) - 2*F(n-3) - F(n-6). - _Gary Detlefs_, Apr 01 2012
   - F(n) = round(phi^(n+1)/(phi+2)). - _Thomas Ordowski_, Apr 20 2012
   - From _Sergei N. Gladkovskii_, Jun 03 2012: (Start)
   - G.f. A(x) = x/(1-x-x^2) = G(0)/sqrt(5) where G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*c^k/G(k+1))) and a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
   - Let E(x) be the e.g.f., i.e.,
   - E(x) = 1*x + 1/2*x^2 + 1/3*x^3 + 1/8*x^4 + 1/24*x^5 + 1/90*x^6 + 13/5040*x^7 + ...; then
   - E(x) = G(0)/sqrt(5); G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*(k+1)*c^k/G(k+1))), where a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
   - (End)
   - From _Hieronymus Fischer_, Nov 30 2012: (Start)
   - Fib(n) = 1 + Sum_{j_1=1..n-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} 1 + ... + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} ... Sum_{j_k=1..j_(k-1)-2} 1, where k = floor((n-1)/2).
   - Example: Fib(6) = 1 + Sum_{j=1..4} 1 + Sum_{j=1..4} Sum_{k=1..(j-2)} 1 + 0 = 1 + (1 + 1 + 1 + 1) + (1 + (1 + 1)) = 8.
   - Fib(n) = Sum_{j=0..k} S(j+1,n-2j), where k = floor((n-1)/2) and the S(j,n) are the n-th j-simplex sums: S(1,n) = 1 is the 1-simplex sum, S(2,n) = Sum_{k=1..n} S(1,k) = 1+1+...+1 = n is the 2-simplex sum, S(3,n) = Sum_{k=1..n} S(2,k) = 1+2+3+...+n is the 3-simplex sum (= triangular numbers = A000217), S(4,n) = Sum_{k=1..n} S(3,k) = 1+3+6+...+n(n+1)/2 is the 4-simplex sum (= tetrahedral numbers = A000292) and so on.
   - Since S(j,n) = binomial(n-2+j,j-1), the formula above equals the well-known binomial formula, essentially. (End)
   - G.f. A(x) = x / (1 - x / (1 - x / (1 + x))). - _Michael Somos_, Jan 04 2013
   - Sum_{n>=1} (-1)^(n-1)/(a(n)*a(n+1)) = 1/phi (phi=golden ratio). - _Vladimir Shevelev_, Feb 22 2013
   - From _Vladimir Shevelev_, Feb 24 2013: (Start)
   - (1) Expression a(n+1) via a(n): a(n+1) = (a(n) + sqrt(5*(a(n))^2 + 4*(-1)^n))/2;
   - (2) Sum_{k=1...n} (-1)^(k-1)/(a(k)*a(k+1)) = a(n)/a(n+1);
   - (3) a(n)/a(n+1) = 1/phi + r(n), where |r(n)| < 1/(a(n+1)*a(n+2)). (End)
   - F(n+1) = F(n)/2 + sqrt((-1)^n + 5*F(n)^2/4), n>=0. F(n+1) = U_n(i/2)/i^n, (U:= Chebyshef 2nd kind). - _Bill Gosper_, Mar 04 2013
   - G.f.: -Q(0) where Q(k) = 1 - (1+x)/(1 - x/(x - 1/Q(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Mar 06 2013
   - G.f.: x-1-1/x + 1/x/Q(0), where Q(k) = 1 - (k+1)*x/(1 - x/(x - (k+1)/Q(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Apr 23 2013
   - G.f.: x*G(0), where G(k)= 1 + x*(1+x)/(1 - x*(1+x)/(x*(1+x) + 1/G(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Jul 08 2013
   - G.f.: x^2 - 1 + 2*x^2/(W(0)-2), where W(k) = 1 + 1/(1 - x*(k + x)/( x*(k+1 + x) + 1/W(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Aug 28 2013
   - G.f.: Q(0) -1, where Q(k) = 1 + x^2 + (k+2)*x -x*(k+1 + x)/Q(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Oct 06 2013
   - Let b(n) = b(n-1) + b(n-2), with b(0) = 0, b(1) = phi. Then, for n>=2, F(n)= floor(b(n-1)) if n is even, F(n) = ceil(b(n-1)), if n is odd, with convergence. - _Richard R. Forberg_, Jan 19 2014
   - a(n) = Sum_{t1*g(1)+t2*g(2)+...+tn*g(n)=n} multinomial(t1+t2 +...+tn,t1,t2,...,tn), where g(k)=2*k-1. - _Mircea Merca_, Feb 27 2014
   - F(n) = round(sqrt(F(n-1)^2 + F(n)^2 + F(n+1)^2)/2), for n > 0. This rule appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Jul 27 2014
   - F(n) = round(2/(1/F(n) + 1/F(n+1) + 1/F(n+2)), for n > 0. This rule also appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Aug 03 2014
   - F(n) = round(1/(Sum_{j>=n+2} 1/F(j))). - _Richard R. Forberg_, Aug 14 2014
   - a(n) = hypergeometric([-n/2+1/2, -n/2+1], [-n+1], -4) for n>=2. - _Peter Luschny_, Sep 19 2014
   - F(n) = (L(n+1)^2 - L(n-1)^2)/(5*L(n)), where L(n) is A000032(n), with a similar inverse relationship. - _Richard R. Forberg_, Nov 17 2014
   - Consider the graph G[1-vertex;1-loop,2-loop] in comment above. Construct the power matrix array T(n,j)=[A^*j]*[S^*(j-1)] where A=(1,1,0,...) and S=(0,1,0,...)(A063524). [* is convolution operation] Define S^*0=I with I=(1,0,...). Then T(n,j) counts n-walks containing (j) loops and a(n-1) = Sum_{j=1...n} T(n,j). - _David Neil McGrath_, Nov 21 2014
   - Define F(-n) to be F(n) for n odd and -F(n) for n even. Then for all n and k, F(n) = F(k)*F(n-k+3) - F(k-1)*F(n-k+2) - F(k-2)*F(n-k) + (-1)^k*F(n-2k+2). - _Charlie Marion_, Dec 04 2014
   - F(n+k)^2 - L(k)*F(n)*F(n+k) + (-1)^k*F(n)^2 = (-1)^n*F(k)^2, if L(k) = A000032(k). - _Alexander Samokrutov_, Jul 20 2015
   - F(2*n) = F(n+1)^2 - F(n-1)^2, similar to Koshy (D) and Forberg 2011, but different. - _Hermann Stamm-Wilbrandt_, Aug 12 2015
   - F(n+1) = ceiling( (1/phi)*Sum_{k=0..n} F(k) ). - _Tom Edgar_, Sep 10 2015
   - a(n) = (L(n-3) + L(n+3))/10 where L(n)=A000032(n). - _J. M. Bergot_, Nov 25 2015
   - From _Bob Selcoe_, Mar 27 2016 (Start):
   - F(n) = (F(2n+k+1) - F(n+1)*F(n+k+1))/F(n+k), k>=0.
   - Thus when k=0: F(n) = sqrt(F(2n+1) - F(n+1)^2).
   - F(n) = cbrt(F(3n) - F(n+1)^3 + F(n-1)^3).
   - F(n+2k) = binomial transform of any subsequence starting with F(n). Example F(6)=8: 1*8 = F(6)=8; 1*8 + 1*13 = F(8)=21; 1*8 + 2*13 + 1*21 = F(10)=55; 1*8 + 3*13 + 3*21 + 1*34 = F(12)=144, etc. This formula applies to Fibonacci-type sequences with any two seed values for a(0) and a(1) (e.g., Lucas sequence A000032: a(0)=2, a(1)=1).
   - (End)
   - F(n) = L(k)*F(n-k) + (-1)^(k+1)*F(n-2k) for all k>=0, where L(k) = A000032(k). - _Anton Zakharov_, Aug 02 2016
   - From _Ilya Gutkovskiy_, Aug 03 2016: (Start)
   - a(n) = F_n(1), where F_n(x) are the Fibonacci polynomials.
   - Inverse binomial transform of A001906.
   - Number of zeros in substitution system {0 -> 11, 1 -> 1010} at step n from initial string "1" (1 -> 1010 -> 101011101011 -> ...) multiplied by 1/A000079(n). (End)

_Cross references_:
   - Cf. <a href="http://oeis.org/A039834">A039834</a> (signed Fibonacci numbers), <a href="http://oeis.org/A001690">A001690</a> (complement), <a href="http://oeis.org/A000213">A000213</a>, <a href="http://oeis.org/A000288">A000288</a>, <a href="http://oeis.org/A000322">A000322</a>, <a href="http://oeis.org/A000383">A000383</a>, <a href="http://oeis.org/A060455">A060455</a>, <a href="http://oeis.org/A030186">A030186</a>, <a href="http://oeis.org/A020695">A020695</a>, <a href="http://oeis.org/A020701">A020701</a>, <a href="http://oeis.org/A071679">A071679</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A100492">A100492</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A101399">A101399</a>, <a href="http://oeis.org/A101400">A101400</a>, <a href="http://oeis.org/A001611">A001611</a>, <a href="http://oeis.org/A000071">A000071</a>, <a href="http://oeis.org/A157725">A157725</a>, <a href="http://oeis.org/A001911">A001911</a>, <a href="http://oeis.org/A157726">A157726</a>, <a href="http://oeis.org/A006327">A006327</a>, <a href="http://oeis.org/A157727">A157727</a>, <a href="http://oeis.org/A157728">A157728</a>, <a href="http://oeis.org/A157729">A157729</a>, <a href="http://oeis.org/A167616">A167616</a>, <a href="http://oeis.org/A059929">A059929</a>, <a href="http://oeis.org/A144152">A144152</a>, <a href="http://oeis.org/A152063">A152063</a>, <a href="http://oeis.org/A114690">A114690</a>, <a href="http://oeis.org/A003893">A003893</a>, <a href="http://oeis.org/A000032">A000032</a>, <a href="http://oeis.org/A060441">A060441</a>, <a href="http://oeis.org/A000930">A000930</a>, <a href="http://oeis.org/A003269">A003269</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A057078">A057078</a>, <a href="http://oeis.org/A007317">A007317</a>, <a href="http://oeis.org/A091867">A091867</a>, <a href="http://oeis.org/A104597">A104597</a>, <a href="http://oeis.org/A249548">A249548</a>, <a href="http://oeis.org/A262342">A262342</a>.
   - First row of arrays <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A234357">A234357</a>. Second row of arrays <a href="http://oeis.org/A099390">A099390</a>, <a href="http://oeis.org/A048887">A048887</a>, and <a href="http://oeis.org/A092921">A092921</a> (k-generalized Fibonacci numbers).
   - a(n) = <a href="http://oeis.org/A094718">A094718</a>(4, n). a(n) = <a href="http://oeis.org/A101220">A101220</a>(0, j, n).
   - a(n) = <a href="http://oeis.org/A090888">A090888</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(1, n-1) = <a href="http://oeis.org/A109754">A109754</a>(0, n) = <a href="http://oeis.org/A109754">A109754</a>(1, n-1), for n > 0.
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.
   - Boustrophedon transforms: <a href="http://oeis.org/A000738">A000738</a>, <a href="http://oeis.org/A000744">A000744</a>.
   - Powers: <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A105317">A105317</a>, <a href="http://oeis.org/A254719">A254719</a>.
   - Numbers of prime factors: <a href="http://oeis.org/A022307">A022307</a> and <a href="http://oeis.org/A038575">A038575</a>.

_Links_:
   - N. J. A. Sloane, <a href="/A000045/b000045.txt">The first 2000 Fibonacci numbers: Table of n, F(n) for n = 0..2000</a>
   - Amazing Mathematical Object Factory, <a href="http://www.schoolnet.ca/vp-pv/amof/e_fiboI.htm">Information on the Fibonacci sequences</a>
   - Matt Anderson, Jeffrey Frazier and Kris Popendorf, <a href="http://library.thinkquest.org/27890/theSeries.html">The Fibonacci series: the section index</a> [broken link]
   - P. G. Anderson, <a href="http://www.cs.rit.edu/~pga/Fibo/fact_sheet.html">Fibonacci Facts</a>
   - Joerg Arndt, <a href="http://www.jjj.de/fxt/#fxtbook">Matters Computational (The Fxtbook)</a>
   - J.-L. Baril, <a href="http://www.combinatorics.org/Volume_18/PDF/v18i1p178.pdf">Classical sequences revisited with permutations avoiding dotted pattern</a>, Electronic Journal of Combinatorics, 18 (2011), #P178.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Barry/barry84.html">A Catalan Transform and Related Transformations on Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.5.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.
   - A. T. Benjamin, A. K. Eustis, M. A. Shattuck, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Shattuck/shattuck20.html">Compression Theorems for Periodic Tilings and Consequences</a>, JIS 12 (2009) 09.6.3
   - E. R. Berlekamp, <a href="/A257113/a257113.pdf">A contribution to mathematical psychometrics</a>, Unpublished Bell Labs Memorandum, Feb 08 1968 [Annotated scanned copy]
   - Marjorie Bicknell and Verner E Hoggatt Jr, <a href="http://www.fq.math.ca/Books/Primer/bicknell6.pdf">To Prove: F(n) Divides F(nk)</a>, A Primer for the Fibonacci Numbers: IX (1973)
   - J. Bodeen, S. Butler, T. Kim, X. Sun, S. Wang, <a href="http://www.combinatorics.org/ojs/index.php/eljc/article/view/v21i1p7">Tiling a strip with triangles</a>, El. J. Combinat. 21 (1) (2014) P1.7
   - H. Bottomley and N. J. A. Sloane, <a href="/A000045/a000045.html">Illustration of initial terms: the Fibonacci tree</a>
   - Brantacan, <a href="http://www.branta.connectfree.co.uk/fibonacci.htm">Fibonacci Numbers</a> [broken link]
   - J. Britton & B. V. Eeckhout, <a href="http://ccins.camosun.bc.ca/~jbritton/fibonacci/jbfibapplet.htm">Fibonacci Interactive</a> [broken link]
   - S. Brlek, E. Duchi, E. Pergola and S. Rinaldi, <a href="http://dx.doi.org/10.1016/j.disc.2004.07.019">On the equivalence problem for succession rules</a>, Discr. Math., 298 (2005), 142-154.
   - N. D. Cahill, J. R. D'Errico, J. P. Spence, <a href="http://www.fq.math.ca/41-1.html">Complex factorizations of the Fibonacci and Lucas numbers</a>, Fib. Quart. 41 (2003) 13.
   - N. D. Cahill and D. A. Narayan, <a href="http://www.fq.math.ca/Papers1/42-3/quartcahill03_2004.pdf">Fibonacci and Lucas Numbers as Tridiagonal Matrix Determinants</a>, Fibonacci Quarterly, 42(3):216-221, 2004.
   - C. K. Caldwell, The Prime Glossary, <a href="http://primes.utm.edu/glossary/page.php/FibonacciNumber.html">Fibonacci number</a>
   - P. J. Cameron, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL3/groups.html">Sequences realized by oligomorphic permutation groups</a>, J. Integ. Seqs. Vol. 3 (2000), #00.1.5.
   - Hongwei Chen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Chen/chen78.html">Evaluations of Some Variant Euler Sums</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.3.
   - J. H. Conway and N. J. A. Sloane, <a href="/A019586/a019586.pdf">Notes on the Para-Fibonacci and related sequences</a>
   - J. H. Conway, Allan Wechsler, Marc LeBrun, Dan Hoey, N. J. A. Sloane, <a href="/A269725/a269725.txt">On Kimberling Sums and Para-Fibonacci Sequences</a>, Correspondence and Postings to Math-Fun Mailing List, Nov 1996 to Jan 1997
   - E. S. Croot, <a href="http://www.math.gatech.edu/~ecroot/recurrence_notes2.pdf">Notes on Linear Recurrence Sequences</a>
   - Paul Cubre, <a href="http://wakespace.lib.wfu.edu/bitstream/handle/10339/37313/Cubre_wfu_0248M_10299.pdf">The Z-densities of the Fibonacci sequence</a>, M. A. Thesis, Wake Forest University, May 2012;
   - C. Dement, <a href="http://mathforum.org/discuss/sci.math/t/622432">Posting to Math Forum</a> [broken link].
   - R. M. Dickau, <a href="http://mathforum.org/advanced/robertd/fibboard.html">Fibonacci numbers</a>
   - R. Doroslovacki, <a href="http://www.emis.de/journals/MV/9434/mv943407.ps">Binary sequences without 011...110 (k-1 1's) for fixed k</a>, Mat. Vesnik 46 (1994), no. 3-4, 93-98.
   - A.-S. Elsenhans and J. Jahnel, <a href="http://www.uni-math.gwdg.de/tschinkel/gauss/Fibon.pdf">The Fibonacci sequence modulo p^2 - an investigation by computer for p < 10^14</a>.
   - Nathaniel D. Emerson, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Emerson/emerson6.html">A Family of Meta-Fibonacci Sequences Defined by Variable-Order Recursions</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.8.
   - Reinhardt Euler, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL8/Euler/euler1.html">The Fibonacci Number of a Grid Graph and a New Class of Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.2.6.
   - G. Everest, A. J. van der Poorten, Y. Puri and T. Ward, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL5/Ward/ward2.html">Integer Sequences and Periodic Points</a>, Journal of Integer Sequences, Vol. 5 (2002), Article 02.2.3
   - G. Everest, A. van der Poorten, I. Shparlinski and T. Ward, Recurrence Sequences, Amer. Math. Soc., 2003; see esp. p. 255. <a href="http://www.ams.org/mathscinet-getitem?mr=1990179">MR1990179</a>
   - Philipp Fahr and Claus Michael Ringel, <a href="http://www.mathematik.uni-bielefeld.de/~ringel/opus/fr-zwei.pdf">Categorification of the Fibonacci Numbers Using Representations of Quivers</a>
   - John Farrier, <a href="http://uploads.neatorama.com/wp-content/uploads/2010/09/1nT6a-500x296.jpg">Fibonacci Pigeons</a> [From Sarah Spolaor, Sep 30 2010]
   - Helaman and Claire Ferguson, <a href="http://www.ams.org/notices/201007/rtx100700840p.pdf">Celebrating Mathematics in Stone and Bronze</a>, Notices of the American Mathematical Society, 57 (2010), 840-850. See page 844
   - Emmanuel Ferrand, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL10/Ferrand/ferrand8.html">Deformations of the Taylor Formula</a>, Journal of Integer Sequences, Vol. 10 (2007), Article 07.1.7.
   - D. Foata and G.-N. Han, <a href="http://www-irma.u-strasbg.fr/~foata/paper/pub71.html">Nombres de Fibonacci et polynomes orthogonaux</a>,
   - I. Galkin, <a href="http://ulcar.uml.edu/~iag/CS/Fibonacci.html">"Fibonacci Numbers Spelled Out"</a>
   - Dale Gerdemann, <a href="http://www.youtube.com/watch?v=YpTSfkqS7zg">Video of Fibonacci tree</a>
   - Dale Gerdemann, <a href="http://bit.ly/lw7pP2">Video of Fibonacci tree(s)</a> (another version)
   - Dale Gerdemann, <a href="http://www.youtube.com/watch?v=AvRdzZMcUrI">Golden Ratio Base Contains Zeckendorf and Negative Indexed Bunder Forms</a>
   - C. J. Glasby, S. P. Glasby, F. Pleijel, <a href="http://dx.doi.org/10.1098/rspb.2008.0418">Worms by number</a>, Proc. Roy. Soc. B, Proc. Biol. Sci. 275 (1647) (2008) 2071-2076.
   - L. Goldsmith, <a href="http://people.bath.ac.uk/ma2lag/fibonaccinumbers.html">The Fibonacci Numbers</a> [broken link]
   - Hank Green, <a href="https://www.youtube.com/watch?v=wTlw7fNcO-0">The Fibonacci Sequence: Nature's Code</a> (2012).
   - M. Griffiths, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL14/Griffiths/griffiths16.html">A Restricted Random Walk defined via a Fibonacci Process</a>, Journal of Integer Sequences, Vol. 14 (2011), #11.5.4.
   - Nancy S. S. Gu, Nelson Y. Li, and Toufik Mansour, <a href="http://dx.doi.org/10.1016/j.disc.2007.04.007">2-Binary trees: bijections and related issues</a>, Discr. Math., 308 (2008), 1209-1221.
   - Guo-Niu Han, <a href="http://www-irma.u-strasbg.fr/~guoniu/papers/p77puzzle.pdf">Enumeration of Standard Puzzles</a> [broken link]
   - Guo-Niu Han, <a href="/A196265/a196265.pdf">Enumeration of Standard Puzzles</a> [Cached copy]
   - S. Happersett, <a href="http://dx.doi.org/10.1080/17513470701210626">Mathematical meditations</a>
   - Brady Haran and Colm Mulcahy, <a href="https://www.youtube.com/watch?v=4izjrtR8Ozg">Little Fibs - Numberphile</a> (2016)
   - A. P. Hillman & G. L. Alexanderson, Algebra Through Problem Solving, Chapter 2 pp. 11-16, <a href="http://education.lanl.gov/RESOURCES/ATPS/CHPTR02/P011.HTM">The Fibonacci and Lucas Numbers</a> [broken link]
   - A. M. Hinz, S. Klavžar, U. Milutinović, C. Petr, <a href="http://dx.doi.org/10.1007/978-3-0348-0237-6">The Tower of Hanoi - Myths and Maths</a>, Birkhäuser 2013. See page 12. <a href="http://tohbook.info">Book's website</a>
   - V. E. Hoggatt and C. T. Long, <a href="http://www.fq.math.ca/Scanned/12-2/hoggatt1.pdf">Divisibility Properties of Generalized Fibonacci Polynomials</a>, Fibonacci Quarterly, 12:113-120, 1974.
   - C. W. Huegy and D. B. West, <a href="http://dx.doi.org/10.1016/S0012-365X(01)00239-4">A Fibonacci tiling of the plane</a>, Discrete Math., 249 (2002), 111-116.
   - INRIA Algorithms Project, <a href="http://algo.inria.fr/ecs/ecs?searchType=1&amp;service=Search&amp;searchTerms=9">Encyclopedia of Combinatorial Structures 9</a> [broken link]
   - Tina Hill Janzen, <a href="http://www.youtube.com/watch?v=2nAycC7sGVI">Fibonacci sequence / Golden scale</a> [broken link]
   - Q.-H. Hou, Z.-W. Sun and H.-M. Wen, <a href="http://arxiv.org/abs/1208.3903">On monotonicity of some combinatorial sequences</a>, arXiv:1208.3903 [math.CO], 2012-2014.
   - James P. Jones, <a href="http://www.fq.math.ca/Scanned/13-1/jones.pdf">Diophantine representation of the Fibonacci numbers</a>, Fibonacci Quarterly 13:1 (1975), pp. 84-88.
   - R. Jovonovic, <a href="http://milan.milanovic.org/math/english/function/function.html">Fibonacci Function Calculator</a> [broken link]
   - R. Jovonovic, <a href="http://milan.milanovic.org/math/english/pdf/Fibonacci.pdf">The relations between the Fibonacci and the Lucas numbers</a> [broken link]
   - R. Jovanovic, <a href="http://milan.milanovic.org/math/Math.php?akcija=SviFibo">First 70 Fibonacci numbers</a> [broken link]
   - S. Kak, <a href="http://arXiv.org/abs/physics/0411195">The Golden Mean and the Physics of Aesthetics</a>, arXiv:physics/0411195 [physics.hist-ph], 2004.
   - Louis H. Kauffman and Pedro Lopes, <a href="http://arXiv.org/abs/0710.3765">Graded forests and rational knots</a>, arXiv:0710.3765 [math.GT], 2007-2009.
   - Blair Kelly, <a href="http://mersennus.net/fibonacci//">Fibonacci and Lucas factorizations</a>
   - Tanya Khovanova, <a href="http://www.tanyakhovanova.com/RecursiveSequences/RecursiveSequences.html">Recursive Sequences</a>
   - C. Kimberling, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL6/Kimberling/kimberling24.html">Matrix Transformations of Integer Sequences</a>, J. Integer Seqs., Vol. 6, 2003.
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/">Fibonacci numbers and the golden section</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibmaths.html">Mathematics of the Fibonacci Series</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibtable.html">Fibonacci numbers with tables of F(0)-F(500).</a>
   - A. Krowne, PlanetMath.org, <a href="http://planetmath.org/encyclopedia/FibonacciNumber.html">Fibonacci sequence</a> [broken link]
   - Hendrik Lenstra, <a href="http://math.berkeley.edu/~hwl/papers/fibo.pdf">Profinite Fibonacci Numbers</a>
   - M. A. Lerma, <a href="http://www.math.northwestern.edu/~mlerma/problem_solving/results/recurrences.pdf">Recurrence Relations</a>
   - D. Litchfield, D. Goldenheim and C. H. Dietrich, <a href="http://scientium.com/diagon_alley/archival/segments/euclid1.htm">Euclid, Fibonacci and Sketchpad</a>, Math. Teacher, 90 (1997). [broken link]
   - F. Luca, V. J. M. Huguet, F. Nicolae, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Mejia/luca31.html">On the Euler Function of Fibonacci Numbers</a>, JIS 12 (2009) 09.6.6
   - B. Malesevic, <a href="http://matematika.etf.bg.ac.yu/publikacije/pub/P09(98)/P09_06.ZIP">Some combinatorial aspects of differential operation composition on the space R^n </a> [broken link].
   - Charles P. McKeague, <a href="http://www.youtube.com/watch?v=_NmSEEhtc1U">Fibonacci numbers from MathTV</a>
   - Graeme McRae, <a href="http://oeis.org/wiki/User:Graeme_McRae/Sum_of_2m_Consecutive_Fibonacci_Numbers">Sum of 2m consecutive Fibonacci numbers</a>
   - D. Merlini, R. Sprugnoli and M. C. Verri, <a href="http://www.dsi.unifi.it/~merlini/tiling.ps">Strip tiling and regular grammars</a>, Theoret. Computer Sci. 242, 1-2 (2000) 109-124. [broken link]
   - D. Merrill, <a href="http://pw1.netcom.com/~merrills/fibphi.html">The Fib-Phi Link Page</a> [broken link]
   - Jean-Christophe Michel, <a href="http://framy.free.fr/fibonacci%20dans%20mandelbrot.htm">Le nombre d'or dans l'ensemble de Mandelbrot</a> (in French, 'The golden number in the Mandelbrot set')
   - Kerry Mitchell, <a href="http://kerrymitchellart.com/articles/Spirolateral-Type_Images_from_Integer_Sequences.pdf">Spirolateral-Type Images from Integer Sequences</a>, 2013
   - H. Mishima, Factorizations of Fibonacci numbers <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha108.htm">n=1..100</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha109.htm">n=101..200</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha110.htm">n=201..300</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha111.htm">n=301..400</a>, <a href="http://www.asahi-net.or.jp/~KC2H-MSM/mathland/matha1/matha112.htm">n=401..480</a>
   - P. Moree, <a href="http://arXiv.org/abs/math.CO/0311205">Convoluted convolved Fibonacci numbers</a>, arXiv:math/0311205 [math.CO], 2003.
   - R. Mullen, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL12/Mullen/mullen2.html">On Determining Paint by Numbers Puzzles with Nonunique Solutions</a>, JIS 12 (2009) 09.6.5
   - Newton Institute, <a href="http://www.newton.cam.ac.uk/wmy2kposters/january">Posters in the London Underground</a>
   - Tony D. Noe and Jonathan Vos Post, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Noe/noe5.html">Primes in Fibonacci n-step and Lucas n-step Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.4.
   - J. J. O'Connor and E. F. Robertson, Mac Tutor History of Mathematics, <a href="http://www-history.mcs.st-andrews.ac.uk/Biographies/Hemchandra.html">Archarya Hemachandra</a>
   - Arzu Özkoç, <a href="http://link.springer.com/article/10.1186/s13662-015-0486-7/fulltext.html">Some algebraic identities on quadra Fibona-Pell integer sequence</a>, Advances in Difference Equations, 2015, 2015:148.
   - Ram Krishna Pandey, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL16/Pandey/pandey7.html">On Some Magnified Fibonacci Numbers Modulo a Lucas Number</a>, Journal of Integer Sequences, Vol. 16 (2013), #13.1.7.
   - J. Patterson, <a href="http://www.bath.ac.uk/~ma1jmp/link.html">The Fibonacci Sequence</a>. [broken link]
   - Ed Pegg, Jr., <a href="http://www.mathpuzzle.com/MAA/07-Sequence%20Pictures/mathgames_12_08_03.html">Sequence Pictures</a>, Math Games column, Dec 08 2003.
   - Ed Pegg, Jr., <a href="/A000043/a000043_2.pdf">Sequence Pictures</a>, Math Games column, Dec 08 2003 [Cached copy, with permission (pdf only)]
   - T. K. Petersen and B. E. Tenner, <a href="http://arxiv.org/abs/1202.4765">The depth of a permutation</a>, arXiv:1202.4765v1 [math.CO], 2012-2014.
   - Ivars Peterson, <a href="http://www.sciencenews.org/articles/20060603/mathtrek.asp">Fibonacci's Missing Flowers</a>.
   - Akos Pinter and Volker Ziegler, <a href="http://arxiv.org/abs/1005.3624">On Arithmetic Progressions in Recurrences - A new characterization of the Fibonacci sequence</a>, arXiv:1005.3624 [math.NT], 2010.
   - Simon Plouffe, Project Gutenberg, <a href="http://ibiblio.org/pub/docs/books/gutenberg/etext01/fbncc10.txt">The First 1001 Fibonacci Numbers</a> [broken link]
   - Project Nayuki, <a href="http://www.nayuki.io/page/fast-fibonacci-algorithms">Fast Fibonacci algorithms</a> (fast doubling is faster than matrix multiplication).
   - S. Rabinowitz, <a href="http://www.mathpropress.com/stan/bibliography/algorithmicFib.pdf">Algorithmic Manipulation of Fibonacci Identities</a> (1996).
   - Arulalan Rajan, R. Vittal Rao, Ashok Rao and H. S. Jamadagni, <a href="http://arxiv.org/abs/1205.5398">Fibonacci Sequence, Recurrence Relations, Discrete Probability Distributions and Linear Convolution</a>, arXiv preprint arXiv:1205.5398 [math.PR], 2012. - From _N. J. A. Sloane_, Oct 23 2012
   - Marc Renault, <a href="http://www.math.temple.edu/~renault/fibonacci/thesis.html">Properties of the Fibonacci sequence under various moduli</a>, MSc Thesis, Wake Forest U, 1996.
   - N. Renton, <a href="http://www.users.bigpond.net.au/renton/903.htm">The fibonacci Series</a>
   - B. Rittaud, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL10/Rittaud2/rittaud11.pdf">On the Average Growth of Random Fibonacci Sequences</a>, Journal of Integer Sequences, 10 (2007), Article 07.2.4.
   - Rosetta Code, <a href="http://rosettacode.org/wiki/Fibonacci_sequence">A collection of codes to compute fibonacci numbers with different computer languages</a>
   - E. S. Rowland, <a href="http://people.hofstra.edu/Eric_Rowland/investigations/fibonacci.html">Fibonacci Sequence Calculator up to n=1474</a>
   - Michelle Rudolph-Lilith, <a href="http://arxiv.org/abs/1508.07894">On the Product Representation of Number Sequences, with Application to the Fibonacci Family</a>, arXiv preprint arXiv:1508.07894, 2015
   - Shiva Samieinia, <a href="http://www.math.su.se/reports/2007/6/">Digital straight line segments and curves</a>. Licentiate Thesis. Stockholm University, Department of Mathematics, Report 2007:6.
   - A. Sapounakis, I. Tasoulas and P. Tsikouras, <a href="https://cs.uwaterloo.ca/journals/JIS/VOL9/Tsikouras/tsikouras67.html">On the Dominance Partial Ordering of Dyck Paths</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.5.
   - D. Schweizer, <a href="http://math.holycross.edu/~davids/fibonacci/fibonacci.html">First 500 Fibonacci Numbers in blocks of 100.</a> [broken link]
   - Mark A. Shattuck, <a href="http://www.emis.de/journals/INTEGERS/papers/j5/j5.Abstract.html">Tiling proofs of some formulas for the Pell numbers of odd index</a>, Integers, 9 (2009), 53-64.
   - Mark A. Shattuck and Carl G. Wagner, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Shattuck/shattuck56.html">Periodicity and Parity Theorems for a Statistic on r-Mino Arrangements</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.3.6.
   - S. Silvia, <a href="http://arttech.about.com/library/weekly/aa060900a_fibonacci_sequence.htm">Fibonacci sequence</a> [broken link]
   - Parmanand Singh, <a href="http://dx.doi.org/10.1016/0315-0860(85)90021-7">The so-called Fibonacci numbers in ancient and medieval India</a>, Historia Mathematica, Volume 12 (3), 1985, 229-244.
   - Jaap Spies, <a href="http://www.jaapspies.nl/oeis/a000045.sage">Sage program for computing A000045</a>
   - Michael Z. Spivey and Laura L. Steil, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Spivey/spivey7.html">The k-Binomial Transforms and the Hankel Transform</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.1.1.
   - Z.-H. Sun, <a href="http://202.195.112.2/xsjl/szh/ConFn.pdf">Congruences For Fibonacci Numbers</a>
   - Roberto Tauraso, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Tauraso/tauraso3.html">A New Domino Tiling Sequence</a>, Journal of Integer Sequences, Vol. 7 (2004), Article 04.2.3.
   - Thesaurus.Maths.org, <a href="http://thesaurus.maths.org/dictionary/map/word/3788">Fibonacci sequence</a> [broken link]
   - K. Tognetti, <a href="/A000045/a000045.pdf">Letter to N. J. A. Sloane (with attachments), May 25 1994</a>
   - K. Tognetti, <a href="/A000045/a000045_2.pdf">The Search for the Golden Sequence</a>, Draft Manuscript, May 25 1994.
   - K. Tognetti, <a href="http://www.austms.org.au/Modules/Fib">Fibonacci-His Rabbits and His Numbers and Kepler</a>
   - Tony van Ravenstein, <a href="/A000045/a000045_1.pdf">The three gap theorem (Steinhaus conjecture)</a>, Journal of the Australian Mathematical Society (Series A) 45.03 (1988): 360-370. [Annotated scanned copy]
   - C. Vila, <a href="http://www.boingboing.net/2010/03/22/dreamlike-animation.html">Nature by numbers</a> (animation).
   - Christobal Vila, <a href="http://wimp.com/naturenumbers/">Nature Numbers</a> (Video related to Fibonacci numbers)
   - N. N. Vorob'ev, <a href="http://eom.springer.de/F/f040020.htm">Fibonacci numbers</a>, Springer's Encyclopaedia of Mathematics.
   - Carl G. Wagner, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL7/Wagner/wagner3.html">Partition Statistics and q-Bell Numbers (q = -1)</a>, J. Integer Seqs., Vol. 7, 2004.
   - Robert Walker, <a href="http://www.youtube.com/watch?v=Wx4ZfuMl-FI&amp;NR=1">Inharmonic "Golden Rhythmicon" - Fibonacci Sequence in Pairs Approaching Golden Ratio - With Bounce</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/FibonacciNumber.html">Fibonacci Number</a>, <a href="http://mathworld.wolfram.com/Double-FreeSet.html">Double-Free Set</a>, <a href="http://mathworld.wolfram.com/Fibonaccin-StepNumber.html">Fibonacci n-Step Number</a>, <a href="http://mathworld.wolfram.com/ResistorNetwork.html">Resistor Network</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/HosoyaIndex.html">Hosoya Index</a>
   - Eric Weisstein's World of Mathematics, <a href="http://mathworld.wolfram.com/PathGraph.html">Path Graph</a>
   - Wikipedia, <a href="http://www.wikipedia.org/wiki/Fibonacci_number">Fibonacci number</a>
   - Wikipedia, <a href="https://en.wikipedia.org/wiki/Cassini_and_Catalan_identities">Cassini and Catalan identities</a>
   - Willem's Fibonacci site, <a href="http://home.zonnet.nl/LeonardEuler/fiboe.htm">Fibonacci</a>
   - Mike Winkler, <a href="http://arxiv.org/abs/1412.0519">On the structure and the behaviour of Collatz 3n + 1 sequences - Finite subsequences and the role of the Fibonacci sequence</a>, arXiv:1412.0519 [math.GM], 2014.
   - Roman Witula, Damian Slota and Edyta Hetmaniok, <a href="http://ami.ektf.hu/uploads/papers/finalpdf/AMI_41_from255to263.pdf">Bridges between different known integer sequences</a>, Annales Mathematicae et Informaticae, 41 (2013) pp. 255-263.
   - Aimei Yu and Xuezheng Lv, <a href="http://dx.doi.org/10.1007/s10910-006-9088-7">The Merrifield-Simmons indices and Hosoya indices of trees with k pendant vertices</a>, J. Math. Chem., Vol. 41 (2007), pp. 33-43. See page 35.
   - Tianping Zhang and Yuankui Ma, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Zhang/zhang56.html">On Generalized Fibonacci Polynomials and Bernoulli Numbers</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.5.3.
   - <a href="/index/Cor#core">Index entries for "core" sequences</a>
   - <a href="/index/Di#divseq">Index to divisibility sequences</a>
   - <a href="/index/Par#partN">Index entries for related partition-counting sequences</a>
   - <a href="/index/Rec#order_02">Index entries for linear recurrences with constant coefficients</a>, signature (1,1).
   - <a href="/index/Tu#2wis">Index entries for two-way infinite sequences</a>

_References_:
   - Abrate, Marco; Barbero, Stefano; Cerruti, Umberto; Murru, Nadir. Colored compositions, Invert operator and elegant compositions with the "black tie". Discrete Math. 335 (2014), 1-7. MR3248794
   - Andrews, George E. Fibonacci numbers and the Rogers-Ramanujan identities. Fibonacci Quart. 42 (2004), no. 1, 3-19. MR2060558(2005b:11161)
   - S. V. Ault and C. Kicey, Counting paths in corridors using circular Pascal arrays, Discrete Mathematics (2014).
   - Mohammad K. Azarian, The Generating Function for the Fibonacci Sequence, Missouri Journal of Mathematical Sciences, Vol. 2, No. 2, Spring 1990, pp. 78-79. Zentralblatt MATH, Zbl 1097.11516.
   - Mohammad K. Azarian, A Generalization of the Climbing Stairs Problem II, Missouri Journal of Mathematical Sciences, Vol. 16, No. 1, Winter 2004, pp. 12-17.
   - Mohammad K. Azarian, Fibonacci Identities as Binomial Sums, International Journal of Contemporary Mathematical Sciences, Vol. 7,  No. 38, 2012, pp. 1871-1876.
   - Mohammad K. Azarian, Fibonacci Identities as Binomial Sums II, International Journal of Contemporary Mathematical Sciences, Vol. 7,  No. 42, 2012, pp. 2053-2059.
   - Mohammad K. Azarian, Identities Involving Lucas or Fibonacci and Lucas Numbers as Binomial Sums, International Journal of Contemporary Mathematical Sciences, Vol. 7, No. 45, 2012, pp. 2221-2227.
   - P. Bachmann, Niedere Zahlentheorie (1902, 1910), reprinted Chelsea, NY, 1968, vol. 2, p. 70.
   - R. B. Banks, Slicing Pizzas, Racing Turtles and Further Adventures in Applied Mathematics, Princeton Univ. Press, 1999. See p. 84.
   - A. T. Benjamin and J. J. Quinn, Proofs that really count: the art of combinatorial proof, M.A.A. 2003, id. 4.
   - Marjorie Bicknell and Verner E Hoggatt, Fibonacci's Problem Book, Fibonacci Association, San Jose, Calif., 1974.
   - A. Cayley, Theorems in Trigonometry and on Partitions, Messenger of Mathematics, 5 (1876), pp. 164, 188 = Mathematical Papers Vol. 10, n. 634, p. 16.
   - D. Chmiela, K. Kaczmarek, R. Witula, Binomials Transformation Formulas of Scaled Fibonacci Numbers (submitted 2012).
   - B. A. Davey and H. A. Priestley, Introduction to Lattices and Order (2nd edition), CUP, 2002. (See Exercise 1.15.)
   - B. Davis, 'The law of first digits' in 'Science Today'(subsequently renamed '2001') March 1980 p. 55, Times of India, Mumbai.
   - S. R. Finch, Mathematical Constants, Cambridge, 2003, Section 1.2.
   - R. P. Grimaldi, Compositions without the summand 1, Proceedings Thirty-second Southeastern International Conference on Combinatorics, Graph Theory and Computing (Baton Rouge, LA, 2001). Congr. Numer. 152 (2001), 33-43.
   - H. Halberstam and K. F. Roth, Sequences, Oxford, 1966; see Appendix.
   - S. Happersett, "Mathematical meditations", Journal of Mathematics and the Arts, 1 (2007), 29 - 33.
   - G. H. Hardy and E. M. Wright, An Introduction to the Theory of Numbers. 3rd ed., Oxford Univ. Press, 1954; see esp. p. 148.
   - J. Hermes, Anzahl der Zerlegungen einer ganzen rationalen Zahl in Summanden, Math. Ann., 45 (1894), 371-380.
   - V. E. Hoggatt, Jr., Fibonacci and Lucas Numbers. Houghton, Boston, MA, 1969.
   - E. Horowitz and S. Sahni, Fundamentals of Data Structures, Computer Science Press, 1976; p. 338.
   - P. W. Kasteleyn, The statistics of dimers on a lattice. I. The number of dimer arrangements on a quadratic lattice, Physica, 27 (1961), 1209-1225.
   - M. Kauers and P. Paule, The Concrete Tetrahedron, Springer 2011, p. 63.
   - C. Kicey and K. Klimko, Some geometry of Pascal's triangle, Pi Mu Epsilon Journal, 13(4):229-245 (2011).
   - D. E. Knuth, The Art of Computer Programming. Addison-Wesley, Reading, MA, Vol. 1, p. 78; Vol. 3, Section 6.2.1.
   - Thomas Koshy, "Fibonacci and Lucas Numbers with Applications", John Wiley and Sons, 2001.
   - Leonardo of Pisa [Leonardo Pisano], Liber Abaci [The Book of Calculation], 1202.
   - Lukovits et al., Nanotubes: Number of Kekulé structures and aromaticity, J. Chem. Inf. Comput. Sci, (2003), vol. 43, 609-614. See eq. 2 on page 610.
   - I. Lukovits and D. Janezic, "Enumeration of conjugated circuits in nanotubes", J. Chem. Inf. Comput. Sci., vol. 44, 410-414 (2004). See Table 1, second column.
   - B. Malesevic: Some combinatorial aspects of differential operation composition on the space R^n, Univ. Beograd, Publ. Elektrotehn. Fak., Ser. Mat. 9 (1998), 29-33.
   - G. Mantel, Resten van wederkeerige Reeksen, Nieuw Archief v. Wiskunde, 2nd series, I (1894), 172-184.
   - C. N. Menhinick, The Fibonacci Resonance and other new Golden Ratio discoveries, Onperson, (2015), pages 200-206.
   - A. Milicevic and N. Trinajstic, "Combinatorial Enumeration in Chemistry", Chem. Modell., Vol. 4, (2006), pp. 405-469.
   - S. Mneimneh, Fibonacci in The Curriculum: Not Just a Bad Recurrence, in Proceeding SIGCSE '15 Proceedings of the 46th ACM Technical Symposium on Computer Science Education, Pages 253-258.
   - Clifford A. Pickover, A Passion for Mathematics, Wiley, 2005; see p. 49.
   - Clifford A. Pickover, The Math Book, From Pythagoras to the 57th Dimension, 250 Milestones in the History of Mathematics, Sterling Publ., NY, 2009, page 274.
   - A. S. Posamentier & I. Lehmann, The Fabulous Fibonacci Numbers, Prometheus Books, Amherst, NY 2007.
   - P. Ribenboim, The New Book of Prime Number Records, Springer, 1996.
   - J. Riordan, An Introduction to Combinatorial Analysis, Princeton University Press, Princeton, NJ, 1978.
   - A. M. Robert, A Course in p-adic Analysis, Springer-Verlag, 2000; p. 213.
   - J. Roberts, Lure of the Integers, Math. Assoc. America, 1992, p. 288.
   - Manfred R. Schroeder, "Number Theory in Science and Communication", 5th ed., Springer-Verlag, 2009
   - L. E. Sigler, Fibonacci's Liber Abaci, Springer, 2003, pp. 404-405 and [26] on p. 627.
   - Simson, [No first name given], An explanation of an obscure passage in Albert Girard's Commentary ..., Phil. Trans. Royal Soc., 10 (1753), 430-433.
   - N. J. A. Sloane, A Handbook of Integer Sequences, Academic Press, 1973 (includes this sequence).
   - N. J. A. Sloane and Simon Plouffe, The Encyclopedia of Integer Sequences, Academic Press, 1995 (includes this sequence).
   - S. Vajda, Fibonacci and Lucas numbers and the Golden Section, Ellis Horwood Ltd., Chichester, 1989.
   - Van Ravenstein, Tony. "The three gap theorem (Steinhaus conjecture)." Journal of the Australian Mathematical Society (Series A) 45.03 (1988): 360-370.
   - N. N. Vorob'ev, Chisla fibonachchi [Russian], Moscow, 1951. English translation, Fibonacci Numbers, Blaisdell, New York and London, 1961.
   - N. N. Vorobiev, Fibonacci Numbers, Birkhauser (Basel;Boston) 2002.
   - A. Y. Z. Wang, P. Wen, On the partial finite sums of the reciprocals of the Fibonacci numbers, Journal of Inequalities and Applications, 2015; http://www.journalofinequalitiesandapplications.com/content/2015/1/73
   - D. Wells, The Penguin Dictionary of Curious and Interesting Numbers, pp. 61-7, Penguin Books 1987.
   - R. Witula, D. Slota, delta-Fibonacci Numbers, Appl. Anal. Discrete Math., 3 (2009), 310-329.
<hr><div align='center'>**<a href='http://oeis.org/A001129'>A001129</a>**: _Iccanobif numbers: reverse digits of two previous terms and add._<br>by _N. J. A. Sloane_.</div>

_Keywords_: `nonn,base,easy,nice`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A001129(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 39 & 124 & 514 & 836 & 1053 & 4139 & 12815 & 61135 & 104937 & 792517 & 1454698 & 9679838
\end{array}
$$




_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A001129">A001129</a>, <a href="http://oeis.org/A014258">A014258</a>-<a href="http://oeis.org/A014260">A014260</a>.
   - See <a href="http://oeis.org/A000045">A000045</a> for the Fibonacci numbers.

_Links_:
   - T. D. Noe and Alois P. Heinz, <a href="/A001129/b001129.txt">Table of n, a(n) for n = 0..1000</a> (first 301 terms from T. D. Noe)

<hr><div align='center'>**<a href='http://oeis.org/A014258'>A014258</a>**: _Iccanobif numbers: add previous two terms and reverse the sum._<br>by _N. J. A. Sloane_.</div>

_Keywords_: `base,nonn,easy`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A014258(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 31 & 93 & 421 & 415 & 638 & 3501 & 9314 & 51821 & 53116 & 739401 & 715297 & 8964541 & 8389769
\end{array}
$$




_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A001129">A001129</a>, <a href="http://oeis.org/A014259">A014259</a>, <a href="http://oeis.org/A014260">A014260</a>.

_Links_:
   - N. J. A. Sloane and Alois P. Heinz, <a href="/A014258/b014258.txt">Table of n, a(n) for n = 0..1000</a> (first 200 terms from N. J. A. Sloane)
   - N. J. A. Sloane, <a href="/transforms.txt">Transforms</a>

<hr><div align='center'>**<a href='http://oeis.org/A003893'>A003893</a>**: _a(n) = Fibonacci(n) mod 10._<br>by _N. J. A. Sloane_, ELIPPER(AT)UOFT02.UTOLEDO.EDU</div>

_Keywords_: `nonn,base,easy`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A003893(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 3 & 1 & 4 & 5 & 9 & 4 & 3 & 7 & 0 & 7 & 7 & 4 & 1
\end{array}
$$


_Comments_:
   - All blocks of 60 successive terms contain 20 even and 40 odd numbers. - _Reinhard Zumkeller_, Apr 09 2005
   - These are the analogs of the Fibonacci numbers in carryless arithmetic mod 10.
   - a(n) = A105471(n) - A105472(n)*10 = A105471(n)/10. - _Reinhard Zumkeller_, Apr 09 2005

_Formulae_:
   - Periodic with period 60 = A001175(10).
   - a(n) = (a(n-1) + a(n-2)) mod 10 for n > 1, a(0) = 0, a(1) = 1. - _Reinhard Zumkeller_, Apr 09 2005

_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A089911">A089911</a>.
   - Cf. <a href="http://oeis.org/A261606">A261606</a>.

_Links_:
   - Vincenzo Librandi, <a href="/A003893/b003893.txt">Table of n, a(n) for n = 0..10000</a>
   - David Applegate, Marc LeBrun and N. J. A. Sloane, <a href="http://neilsloane.com/doc/carry1.pdf">Carryless Arithmetic (I): The Mod 10 Version</a>.
   - Gregory P. Dresden, <a href="http://home.wlu.edu/~dresdeng/papers/lnzd2a.pdf">Three transcendental numbers from the last non-zero digits of n^n, F_n and n!</a>, Math. Mag., pp. 96-105, vol. 81, 2008.
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fibmaths.html">The Mathematical Magic of the Fibonacci Numbers</a>
   - <a href="/index/Fi#final">Index entries for sequences related to final digits of numbers</a>
   - <a href="/index/Ca#CARRYLESS">Index entries for sequences related to carryless arithmetic</a>
   - <a href="/index/Rec#order_39">Index entries for linear recurrences with constant coefficients</a>, signature (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 1, 0, 0, -1, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1).

_References_:
   - G. Marsaglia, The mathematics of random number generators, pp. 73-90 of S. A. Burr, ed., The Unreasonable Effectiveness of Number Theory, Proc. Sympos. Appl. Math., 46 (1992). Amer. Math. Soc.
<hr><div align='center'>**<a href='http://oeis.org/A014260'>A014260</a>**: _Iccanobif numbers: add a(n-1) to reversal of a(n-2)._<br>by _N. J. A. Sloane_.</div>

_Keywords_: `base,nonn,easy`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A014260(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 52 & 64 & 89 & 135 & 233 & 764 & 1096 & 1563 & 8464 & 12115 & 16763
\end{array}
$$




_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A001129">A001129</a>, <a href="http://oeis.org/A014258">A014258</a>, <a href="http://oeis.org/A014259">A014259</a>.
   - See <a href="http://oeis.org/A000045">A000045</a> for the Fibonacci numbers.

_Links_:
   - Alois P. Heinz, <a href="/A014260/b014260.txt">Table of n, a(n) for n = 0..1000</a>

<hr><div align='center'>**<a href='http://oeis.org/A004090'>A004090</a>**: _Sum of digits of Fibonacci numbers._<br>by _N. J. A. Sloane_</div>

_Keywords_: `nonn,base,easy`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A004090(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 4 & 3 & 7 & 10 & 17 & 9 & 8 & 17 & 7 & 24 & 22 & 19 & 14
\end{array}
$$


_Comments_:
   - a(n) and Fib(n) are congruent modulo 9 which implies that (a(n) mod 9) is equal to (Fib(n) mod 9) A007887(n). Thus (a(n) mod 9) is periodic with the Pisano period A001175(9) = 24. - _Hieronymus Fischer_, Jun 25 2007
   - It appears that a(n)-n stays negative for n > 5832, which explains why A020995 is finite. - _T. D. Noe_, Mar 19 2012

_Formulae_:
   - a(n) = Fib(n) - 9*sum{k>0, floor(Fib(n)/10^k)}. - _Hieronymus Fischer_, Jun 25 2007
   - a(n) = A007953(A000045(n)). - _Reinhard Zumkeller_, Nov 17 2014

_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A030132">A030132</a>.
   - Cf. <a href="http://oeis.org/A007953">A007953</a>, <a href="http://oeis.org/A246558">A246558</a>.
   - Cf. <a href="http://oeis.org/A261587">A261587</a>.
   - Cf. <a href="http://oeis.org/A068500">A068500</a>.

_Links_:
   - T. D. Noe, <a href="/A004090/b004090.txt">Table of n, a(n) for n = 0..10000</a>
   - T. D. Noe, <a href="/A004090/a004090_1.gif">Plot of a(n)-n for n = 0..100000</a>

<hr><div align='center'>**<a href='http://oeis.org/A008963'>A008963</a>**: _Initial digit of Fibonacci number F(n)._<br>by _N. J. A. Sloane_.</div>

_Keywords_: `nonn,base,easy,changed`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A008963(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 1 & 2 & 3 & 5 & 8 & 1 & 2 & 3 & 6 & 9 & 1 & 2 & 4
\end{array}
$$


_Comments_:
   - Benford's law applies since the Fibonacci sequence is of exponential growth: P(d)=log_10(1+1/d), in fact among first 5000 values the digit d=1 appears 1505 times, while 5000*P(1) is about 1505.15. [_Carmine Suriano_, Feb 14 2011]


_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A003893">A003893</a> (final digit).
   - Cf. <a href="http://oeis.org/A000030">A000030</a>, <a href="http://oeis.org/A261607">A261607</a>.

_Links_:
   - T. D. Noe, <a href="/A008963/b008963.txt">Table of n, a(n) for n = 0..1000</a>
   - Wikipedia, <a href="http://en.wikipedia.org/wiki/Benford&#39;s_law">Benford's law</a>

<hr><div align='center'>**<a href='http://oeis.org/A007059'>A007059</a>**: _Number of balanced ordered trees with n nodes._<br>by _N. J. A. Sloane_, _Mira Bernstein_, R. Kemp</div>

_Keywords_: `nonn,nice,easy`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A007059(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 14 & 24 & 43 & 77 & 140 & 256 & 472 & 874 & 1628 & 3045 & 5719 & 10780 & 20388
\end{array}
$$


_Comments_:
   - Essentially the same as A079500: a(0)=0 and a(n)=A079500(n-1) for n>=1.
   - Diagonal sums of the "postage stamp" array: for rows n >= -1, column m >= 0 is given by F(n,m)=F(n-1,m)+F(n-2,m)+...+F(n-m,m) with F(0,m)=1 (m >= 0), F(n,m)=0 (n<0) and F(n,0)=0 (n>0). (Rows indicate the required sum, columns indicate the integers available {0,...,m}, entries F(n,m) indicate number of ordered ways sum can be achieved (eg n=3, m=2: 3=1+1+1=1+2=2+1 so F(3,2)=3 ways)). - Richard L. Ollerton (r.ollerton(AT)uws.edu.au)
   - Conjecture: for n>0 a(n+1) is the number of "numbral" divisors of (4^n-1)/3 = A002450(n) (see A048888 for the definition of numbral arithmetic). This has been verified computationally up to n=15. - _John W. Layman_, Dec 18 2001. This conjecture follows immediately from Proposition 2.3 of Frosini and Rinaldi. - _N. J. A. Sloane_, Apr 29 2011.
   - Also number of Dyck paths of semi-length n-1 with all peaks at the same height. (not mentioned in Frosini reference) - _David Scambler_, Nov 19 2010
   - For n>=1, a(n) is the number of compositions of n where all parts are smaller than the first part, see example. For n>=1, a(n-1) = A079500(n) is the number of compositions of n where no part exceeds the first part, see the example in A079500. [_Joerg Arndt_, Dec 29 2012]

_Formulae_:
   - Define generalized Fibonacci numbers by Sum_{h>=0} F(p, h)z^n = z^(p-1)(1-z)/(1-2z+z^p+1). Then a(n) = 1+Sum{2<=h<=n} F(h-1, n-2).
   - G.f.: Sum_{k>0} x^k*(1-2*x+x^2+(1-x)*x^(k+1))/(1-2*x+x^(k+1)). - _Vladeta Jovovic_, Feb 25 2003
   - G.f.: -(1+x^2+ 1/(x-1) )*( 1 + x*(x-1)^3*(1-x+x^3)/( Q(0)- x*(x-1)^3*(1-x+x^3)) ), where Q(k) = (x+1)*(2*x-1)*(1-x)^2 + x^(k+2)*(x+x^2+x^3-2*x^4-1 - x^(k+3) + x^(k+5)) - x*(-1+2*x-x^(k+3))*(1-2*x+x^2+x^(k+4)-x^(k+5))*(-1+4*x-5*x^2+2*x^3 - x^(k+2)- x^(k+5) + 2*x^(k+3) - x^(2*k+5) + x^(2*k+6))/Q(k+1) ; (continued fraction). - _Sergei N. Gladkovskii_, Dec 14 2013

_Cross references_:
   - Cf. <a href="http://oeis.org/A048888">A048888</a>.

_Links_:
   - T. D. Noe and Alois P. Heinz, <a href="/A007059/b007059.txt">Table of n, a(n) for n = 0..1000</a> (first 401 terms from T. D. Noe)
   - A. Frosini and S. Rinaldi, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Frosini/fros2.html">On the Sequence A079500 and Its Combinatorial Interpretations</a>, J. Integer Seq., Vol. 9 (2006), Article 06.3.1.
   - R. Kemp, <a href="http://dx.doi.org/10.1002/rsa.3240050111">Balanced ordered trees</a>, Random Structures Algorithms, 5 (1994), pp. 99-121.
   - <a href="/index/Tra#trees">Index entries for sequences related to trees</a>

_References_:
   - Arnold Knopfmacher and Neville Robbins, Compositions with parts constrained by the leading summand, Ars Combin. 76 (2005), 287-295.
   - N. J. A. Sloane and Simon Plouffe, The Encyclopedia of Integer Sequences, Academic Press, 1995 (includes this sequence).
<hr><div align='center'>**<a href='http://oeis.org/A089911'>A089911</a>**: _a(n) = Fibonacci(n) mod 12._<br>by _Casey Mongoven_, Nov 14 2003</div>

_Keywords_: `nonn,hear,easy`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A089911(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 1 & 9 & 10 & 7 & 5 & 0 & 5 & 5 & 10 & 3 & 1 & 4 & 5
\end{array}
$$


_Comments_:
   - Sequence has been applied by several composers to pitch structure in 12 tone equal temperament. The complete Fibonacci mod 12 system (a set of 10 periodic sequences) exhausts all possible ordered dyads; that is, every possible combination of two pitches is found in these sets.
   - a(A008594(n)) = 0; a(A227144(n)) = 1; a(3*A047522(n)) = 2; a(A017569(n)) = a(2*A016933(n)) = a(4*A016777(n)) = 3; a(2*A017629(n)) = a(3*A017137(n)) = a(6*A004767(n)) = 4; a(A227146(n)) = 5; a(nonexistent) = 6; a(2*A017581(n)) = 7; a(2*A017557(n)) = a(4*A016813(n)) = 8; a(A017617(n)) = a(2*A016957(n)) = a(4*A016789(n)) = 9; a(3*A047621(n)) = 10; a(2*A017653(n)) = 11. - _Reinhard Zumkeller_, Jul 05 2013

_Formulae_:
   - Has period of 24, restricted period 12 and multiplier 5.
   - F(n) mod 12 = F(n-1) + F(n-2), F(0) = 0, F(1) = 1.

_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a>, <a href="http://oeis.org/A001175">A001175</a>, <a href="http://oeis.org/A003893">A003893</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/A089911/b089911.txt">Table of n, a(n) for n = 0..1199</a>
   - Ron Knott, <a href="http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/fib.html">Fibonacci Numbers and the Golden Section</a>
   - M. Renault, <a href="http://www.math.temple.edu/~renault/fibonacci/fib.html">The Fibonacci Sequence Modulo M</a>
   - A. P. Shah, <a href="http://www.fq.math.ca/Scanned/6-2/shah.pdf">Fibonacci Sequence Modulo m</a>, Fibonacci Quarterly, Vol.6, No.2 (1968), 139-141.
   - D. D. Wall, <a href="http://www.jstor.org/stable/2309169">Fibonacci series modulo m</a>, Amer. Math. Monthly, 67 (1960), 525-532.
   - <a href="/index/Rec#order_22">Index entries for linear recurrences with constant coefficients</a>, signature (1, 0, -1, 1, 0, -1, 1, 0, -1, 1, 0, -1, 1, 0, -1, 1, 0, -1, 1, 0, -1, 1).

<hr><div align='center'>**<a href='http://oeis.org/A030132'>A030132</a>**: _Digital root of Fibonacci(n)._<br>by youngelder(AT)webtv.net (Ana)</div>

_Keywords_: `nonn,base,easy,nice`

_Data_:

$$
\begin{array}{c|cccccccccccccccccccc}
n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 & 15 & 16 & 17 & 18 & 19 \\
\hline
A030132(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 4 & 3 & 7 & 1 & 8 & 9 & 8 & 8 & 7 & 6 & 4 & 1 & 5
\end{array}
$$


_Comments_:
   - Any initial pair (a(0), a(1)) of nonzero single-digit numbers enters a cycle of length 24, except for (3, 3), which enters a cycle of length 8 and (9, 9), which is immediately periodic of period length 1. - _Jonathan Vos Post_, Dec 29 2005
   - First term that differs from A004090 is a(10). In general, all terms of A004090 having one digit are the same in this sequence. - _Alonso del Arte_, Sep 16 2012

_Formulae_:
   - a(n + 1) = sum of digits of (a(n) + a(n - 1)).
   - Periodic with period 24 = A001175(9) given by {1, 1, 2, 3, 5, 8, 4, 3, 7, 1, 8, 9, 8, 8, 7, 6, 4, 1, 5, 6, 2, 8, 1, 9}
   - a(n) + a(n + 1) = A010077(n + 4); a(A017641(n)) = 9. - _Reinhard Zumkeller_, Jul 04 2007
   - G.f. x*( -1 -x -2*x^2 -3*x^3 -5*x^4 -8*x^5 -4*x^6 -3*x^7 -7*x^8 -x^9 -8*x^10 -9*x^11 -8*x^12 -8*x^13 -7*x^14 -6*x^15 -4*x^16 -x^17 -5*x^18 -6*x^19 -2*x^20 -8*x^21 -x^22 -9*x^23 ) / ( (x-1) *(1+x+x^2) *(1+x) *(1-x+x^2) *(1+x^2) *(x^4-x^2+1) *(1+x^4) *(x^8-x^4+1) ). - _R. J. Mathar_, Feb 08 2013

_Cross references_:
   - Cf. <a href="http://oeis.org/A000045">A000045</a> (Fibonacci numbers), <a href="http://oeis.org/A010888">A010888</a> (digital roots), <a href="http://oeis.org/A004090">A004090</a>, <a href="http://oeis.org/A007953">A007953</a>, <a href="http://oeis.org/A030133">A030133</a>.
   - Cf. <a href="http://oeis.org/A065076">A065076</a>.

_Links_:
   - Reinhard Zumkeller, <a href="/A030132/b030132.txt">Table of n, a(n) for n = 0..10000</a>
   - Colm Mulcahy, <a href="http://www.maa.org/columns/colm/cardcolm200706.html">Gibonacci Bracelets</a>.
   - Marc Renault, <a href="http://webspace.ship.edu/msrenault/fibonacci/fib.htm">The Fibonacci sequence modulo m</a>
   - <a href="/index/Rec#order_26">Index entries for linear recurrences with constant coefficients</a>, signature (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1).

_References_:
   - S. Marivani and others, Problem 10974, Amer. Math. Monthly, 111 (No. 7, 2004), 628.

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Notebook</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="mailto:massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.